In [12]:
from datetime import datetime
from itertools import combinations
import os
import re

import pandas as pd
import polars as pl
import numpy as np
import matplotlib as plt
import seaborn as sns
import lightgbm as lgb
import sklearn as sk
import mlflow
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.feature_selection import RFECV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

import data_proc as dp

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

# If out of ideas
you can always try submitting:  
removing all the correlated pairs   
removing cat cols with single val  
adding nan indicators
filling 0 where it makes sense

In [2]:
REMOTE_TRACKING_IP = os.getenv("REMOTE_IP", "localhost")
MLFLOW_TRACKING_URI = f"http://{REMOTE_TRACKING_IP}:5000"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

CREDIT_EXPERIMENT_NAME = "credit-score-cv"
EXPERIMENT_NAME = "chosen-models-credit"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(CREDIT_EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/218050481937314459', creation_time=1713079301255, experiment_id='218050481937314459', last_update_time=1713079301255, lifecycle_stage='active', name='credit-score-cv', tags={}>

In [5]:
(base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
cat_cols_base = list(X_train.select_dtypes("category").columns)

In [7]:
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base, free_raw_data=False)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base, free_raw_data=False)
test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base, free_raw_data=False)

In [3]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    higher_better = True
    metric_name="gini_stability"
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

In [6]:
def experiment1(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)
            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))
        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [4]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}


# Experiment 1
Cleaned dataset, categorical variables with 1 value and highly correlated numerical features (pearson > 0.95) removed
rare cat values groupped into 'other' value

In [10]:

bst, eval_res, test_auc, test_gini = experiment1(params)

/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.781355
[100]	eval's auc: 0.799777
[150]	eval's auc: 0.80836
[200]	eval's auc: 0.813556
[250]	eval's auc: 0.816771
[300]	eval's auc: 0.819447
[350]	eval's auc: 0.821402
[400]	eval's auc: 0.822754
[450]	eval's auc: 0.82401
[500]	eval's auc: 0.824911
[550]	eval's auc: 0.82566
[600]	eval's auc: 0.826497
[650]	eval's auc: 0.826989
[700]	eval's auc: 0.827606
[750]	eval's auc: 0.828107
[800]	eval's auc: 0.828609
[850]	eval's auc: 0.829059
[900]	eval's auc: 0.829393
[950]	eval's auc: 0.829766
[1000]	eval's auc: 0.83005
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.83005


/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.779982
[100]	eval's auc: 0.800477
[150]	eval's auc: 0.808933
[200]	eval's auc: 0.814302
[250]	eval's auc: 0.817844
[300]	eval's auc: 0.820429
[350]	eval's auc: 0.822063
[400]	eval's auc: 0.823757
[450]	eval's auc: 0.824961
[500]	eval's auc: 0.826173
[550]	eval's auc: 0.826986
[600]	eval's auc: 0.827776
[650]	eval's auc: 0.828443
[700]	eval's auc: 0.829032
[750]	eval's auc: 0.829492
[800]	eval's auc: 0.830006
[850]	eval's auc: 0.830404
[900]	eval's auc: 0.830812
[950]	eval's auc: 0.831125
[1000]	eval's auc: 0.83133
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.83133


/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.783786
[100]	eval's auc: 0.803775
[150]	eval's auc: 0.813076
[200]	eval's auc: 0.817836
[250]	eval's auc: 0.821221
[300]	eval's auc: 0.82369
[350]	eval's auc: 0.825415
[400]	eval's auc: 0.827041
[450]	eval's auc: 0.828363
[500]	eval's auc: 0.829567
[550]	eval's auc: 0.830244
[600]	eval's auc: 0.830963
[650]	eval's auc: 0.831681
[700]	eval's auc: 0.832405
[750]	eval's auc: 0.832781
Early stopping, best iteration is:
[765]	eval's auc: 0.832894


/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.782092
[100]	eval's auc: 0.801257
[150]	eval's auc: 0.80969
[200]	eval's auc: 0.814669
[250]	eval's auc: 0.817888
[300]	eval's auc: 0.820331
[350]	eval's auc: 0.821853
[400]	eval's auc: 0.823184
[450]	eval's auc: 0.824253
[500]	eval's auc: 0.825225
[550]	eval's auc: 0.826043
[600]	eval's auc: 0.826795
[650]	eval's auc: 0.827368
[700]	eval's auc: 0.827869
[750]	eval's auc: 0.828306
[800]	eval's auc: 0.82874
[850]	eval's auc: 0.829333
[900]	eval's auc: 0.829667
[950]	eval's auc: 0.829909
[1000]	eval's auc: 0.830201
Did not meet early stopping. Best iteration is:
[999]	eval's auc: 0.830209


/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.782582
[100]	eval's auc: 0.80152
[150]	eval's auc: 0.810042
[200]	eval's auc: 0.815071
[250]	eval's auc: 0.818453
[300]	eval's auc: 0.820836
[350]	eval's auc: 0.822664
[400]	eval's auc: 0.824102
[450]	eval's auc: 0.825311
[500]	eval's auc: 0.82634
[550]	eval's auc: 0.827389
[600]	eval's auc: 0.828068
[650]	eval's auc: 0.828669
[700]	eval's auc: 0.829427
[750]	eval's auc: 0.829987
[800]	eval's auc: 0.830447
[850]	eval's auc: 0.830868
[900]	eval's auc: 0.831314
[950]	eval's auc: 0.831824
[1000]	eval's auc: 0.832109
Did not meet early stopping. Best iteration is:
[998]	eval's auc: 0.83211
results:  0.8313144765244568 0.8306314690501406 0.6172219369994861


In [14]:
with mlflow.start_run():
    mlflow.log_dict(
        {
            "eval_auc": eval_res,
            "test_auc": test_auc,
            "test_gini": test_gini
        },  
        "experiment1.json"
    )

# Experiment 2
Check if removing cat columns with 1 value improve anything

In [5]:
def experiment2(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            
            X_train, cols_to_rm = dp.remove_single_val_cols(X_train, cat_cols_base)
            X_valid = X_valid.drop(columns=cols_to_rm)
            X_test = X_test.drop(columns=cols_to_rm)
            cat_cols_base = list(X_train.select_dtypes("category").columns)

            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)

            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))

        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "Check if removing cat columns with 1 value improve anything"
            },  
            "experiment2.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [6]:
bst, _, _, _ = experiment2(params)

bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L


/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.780072
[100]	eval's auc: 0.800172
[150]	eval's auc: 0.808707
[200]	eval's auc: 0.814084
[250]	eval's auc: 0.817302
[300]	eval's auc: 0.81967
[350]	eval's auc: 0.821319
[400]	eval's auc: 0.822917
[450]	eval's auc: 0.824217
[500]	eval's auc: 0.825172
[550]	eval's auc: 0.826007
[600]	eval's auc: 0.826769
[650]	eval's auc: 0.827453
[700]	eval's auc: 0.827847
[750]	eval's auc: 0.828356
[800]	eval's auc: 0.828942
[850]	eval's auc: 0.829369
[900]	eval's auc: 0.829631
[950]	eval's auc: 0.829935
[1000]	eval's auc: 0.830177
Did not meet early stopping. Best iteration is:
[999]	eval's auc: 0.830179
bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L


/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.778786
[100]	eval's auc: 0.798706
[150]	eval's auc: 0.80781
[200]	eval's auc: 0.813254
[250]	eval's auc: 0.816928
[300]	eval's auc: 0.819283
[350]	eval's auc: 0.820988
[400]	eval's auc: 0.822538
[450]	eval's auc: 0.823685
[500]	eval's auc: 0.824876
[550]	eval's auc: 0.82579
[600]	eval's auc: 0.826604
[650]	eval's auc: 0.827358
[700]	eval's auc: 0.828001
[750]	eval's auc: 0.828524
[800]	eval's auc: 0.828994
[850]	eval's auc: 0.829401
Early stopping, best iteration is:
[841]	eval's auc: 0.829401
bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L


...

Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.779748
[100]	eval's auc: 0.799445
[150]	eval's auc: 0.808222
[200]	eval's auc: 0.813553
[250]	eval's auc: 0.816794
[300]	eval's auc: 0.819069
[350]	eval's auc: 0.820944
[400]	eval's auc: 0.8224
[450]	eval's auc: 0.823538
[500]	eval's auc: 0.824584
[550]	eval's auc: 0.825515
[600]	eval's auc: 0.826261
[650]	eval's auc: 0.82671
[700]	eval's auc: 0.8273
[750]	eval's auc: 0.82791
[800]	eval's auc: 0.828338
[850]	eval's auc: 0.828772
[900]	eval's auc: 0.829077
Early stopping, best iteration is:
[925]	eval's auc: 0.829179
bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L


/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.784699
[100]	eval's auc: 0.803109
[150]	eval's auc: 0.811466
[200]	eval's auc: 0.816602
[250]	eval's auc: 0.81994
[300]	eval's auc: 0.822541
[350]	eval's auc: 0.824467
[400]	eval's auc: 0.82603
[450]	eval's auc: 0.827215
[500]	eval's auc: 0.828251
[550]	eval's auc: 0.829142
[600]	eval's auc: 0.829939
[650]	eval's auc: 0.830759
[700]	eval's auc: 0.831367
[750]	eval's auc: 0.831941
[800]	eval's auc: 0.832212
[850]	eval's auc: 0.832649
[900]	eval's auc: 0.83303
Early stopping, best iteration is:
[904]	eval's auc: 0.833056
bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L


/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.786084
[100]	eval's auc: 0.80605
[150]	eval's auc: 0.815478
[200]	eval's auc: 0.820222
[250]	eval's auc: 0.823494
[300]	eval's auc: 0.825564
[350]	eval's auc: 0.827262
[400]	eval's auc: 0.828701
[450]	eval's auc: 0.829567
[500]	eval's auc: 0.830431
[550]	eval's auc: 0.831335
[600]	eval's auc: 0.832063
[650]	eval's auc: 0.832621
[700]	eval's auc: 0.833294
[750]	eval's auc: 0.833695
[800]	eval's auc: 0.834053
[850]	eval's auc: 0.834435
[900]	eval's auc: 0.834824
[950]	eval's auc: 0.835163
[1000]	eval's auc: 0.835622
Did not meet early stopping. Best iteration is:
[993]	eval's auc: 0.835642
results:  0.831485398965562 0.8296554890581076 0.6224366142149258


# Experiment 3
See if thowing rare category values into "others" value improves anything

In [10]:
def experiment3(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            
            dp.rare_values_to_others(X_train, cat_cols_base)
            dp.rare_values_to_others(X_valid, cat_cols_base)
            dp.rare_values_to_others(X_test, cat_cols_base)
            
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)

            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))

        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "See if thowing rare category values into others value improves anything"
            },  
            "experiment3.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [11]:
bst, _, _, _ = experiment3(params)

lastapprcommoditycat_1041M P201_108_190
lastcancelreason_561M P91_110_150
lastrejectcommoditycat_161M P151_9_55
lastrejectcommodtypec_5251769M P34_45_168
lastrejectreason_759M P185_59_178
lastrejectreasonclient_4145040M P59_114_135
max_cancelreason_3545846M P32_86_86
max_rejectreason_755M P53_10_15
max_rejectreasonclient_4145042M P30_86_84
min_cancelreason_3545846M P91_110_150
min_rejectreasonclient_4145042M P59_114_135
first_cancelreason_3545846M P91_110_150
first_rejectreason_755M P185_59_178
first_rejectreasonclient_4145042M P59_114_135
last_cancelreason_3545846M P32_86_86
last_rejectreason_755M P53_10_15
last_rejectreasonclient_4145042M P64_121_167
mode_cancelreason_3545846M P32_86_86
mode_rejectreason_755M P69_72_116
mode_rejectreasonclient_4145042M P59_114_135
last_status_219L P
max_empladdr_district_926M P121_51_168
max_empladdr_zipcode_114M P129_32_166
last_empladdr_district_926M P121_51_168
last_empladdr_zipcode_114M P129_32_166
max_empl_industry_691L ARMY_POLICE
max_incometyp

/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.791492
[100]	eval's auc: 0.809268
[150]	eval's auc: 0.81678
[200]	eval's auc: 0.821759
[250]	eval's auc: 0.824892
[300]	eval's auc: 0.827044
[350]	eval's auc: 0.828732
[400]	eval's auc: 0.830055
[450]	eval's auc: 0.831161
[500]	eval's auc: 0.832029
[550]	eval's auc: 0.832774
[600]	eval's auc: 0.833723
[650]	eval's auc: 0.834338
[700]	eval's auc: 0.835007
[750]	eval's auc: 0.835474
Early stopping, best iteration is:
[785]	eval's auc: 0.835733
lastapprcommoditycat_1041M P201_108_190
lastcancelreason_561M P91_110_150
lastrejectcommoditycat_161M P38_69_128
lastrejectcommodtypec_5251769M P184_3_97
lastrejectreasonclient_4145040M P204_22_168
max_cancelreason_3545846M P32_86_86
max_rejectreason_755M P53_10_15
max_rejectreasonclient_4145042M P30_86_84
min_cancelreason_3545846M P91_110_150
min_rejectreason_755M P185_59_178
min_rejectreasonclient_4145042M P59_114_135
first_cancelreason_3545846M P91_110_150
first_reje

/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.784082
[100]	eval's auc: 0.802896
[150]	eval's auc: 0.811722
[200]	eval's auc: 0.816431
[250]	eval's auc: 0.819616
[300]	eval's auc: 0.821991
[350]	eval's auc: 0.823707
[400]	eval's auc: 0.825191
[450]	eval's auc: 0.826384
[500]	eval's auc: 0.82742
[550]	eval's auc: 0.828158
[600]	eval's auc: 0.82889
[650]	eval's auc: 0.829691
[700]	eval's auc: 0.830196
[750]	eval's auc: 0.830777
[800]	eval's auc: 0.831148
[850]	eval's auc: 0.831568
[900]	eval's auc: 0.831988
[950]	eval's auc: 0.832351
[1000]	eval's auc: 0.832661
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.832661
lastapprcommoditycat_1041M P201_108_190
lastcancelreason_561M P91_110_150
lastrejectcommoditycat_161M P38_69_128
lastrejectcommodtypec_5251769M P185_44_170
lastrejectreason_759M P185_59_178
lastrejectreasonclient_4145040M P204_22_168
lastst_736L P
max_cancelreason_3545846M P32_86_86
max_rejectreason_755M P53_10_15
max_rejec

...

Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.784582
[100]	eval's auc: 0.804157
[150]	eval's auc: 0.812941
[200]	eval's auc: 0.818459
[250]	eval's auc: 0.821803
[300]	eval's auc: 0.823858
[350]	eval's auc: 0.825949
[400]	eval's auc: 0.827341
[450]	eval's auc: 0.828423
[500]	eval's auc: 0.829479
[550]	eval's auc: 0.830347
[600]	eval's auc: 0.83109
[650]	eval's auc: 0.831771
[700]	eval's auc: 0.83231
[750]	eval's auc: 0.832836
[800]	eval's auc: 0.833489
[850]	eval's auc: 0.833841
[900]	eval's auc: 0.834125
[950]	eval's auc: 0.834446
[1000]	eval's auc: 0.834717
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.834717
lastapprcommoditycat_1041M P201_108_190
lastcancelreason_561M P91_110_150
lastrejectcommoditycat_161M P151_9_55
lastrejectcommodtypec_5251769M P34_45_168
lastrejectreason_759M P185_59_178
lastrejectreasonclient_4145040M P59_114_135
lastst_736L R
max_cancelreason_3545846M P32_86_86
max_rejectreason_755M P53_10_15
max_rejectr

/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.783351
[100]	eval's auc: 0.80223
[150]	eval's auc: 0.810932
[200]	eval's auc: 0.81569
[250]	eval's auc: 0.818786
[300]	eval's auc: 0.821033
[350]	eval's auc: 0.822933
[400]	eval's auc: 0.824201
[450]	eval's auc: 0.825317
[500]	eval's auc: 0.82615
[550]	eval's auc: 0.826885
[600]	eval's auc: 0.827529
[650]	eval's auc: 0.828113
[700]	eval's auc: 0.828555
[750]	eval's auc: 0.829021
[800]	eval's auc: 0.829356
[850]	eval's auc: 0.829647
[900]	eval's auc: 0.829936
[950]	eval's auc: 0.830292
[1000]	eval's auc: 0.830595
Did not meet early stopping. Best iteration is:
[999]	eval's auc: 0.830596
lastapprcommoditycat_1041M P201_108_190
lastcancelreason_561M P91_110_150
lastrejectcommoditycat_161M P38_69_128
lastrejectcommodtypec_5251769M P132_130_54
lastrejectreason_759M P185_59_178
lastrejectreasonclient_4145040M P204_22_168
max_cancelreason_3545846M P32_86_86
max_rejectreason_755M P53_10_15
max_rejectreasonclient_41

/home/wojciechostrowski/.cache/pypoetry/virtualenvs/kaggle-home-credit-n3bSIZTJ-py3.11/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.782453
[100]	eval's auc: 0.802037
[150]	eval's auc: 0.810355
[200]	eval's auc: 0.815399
[250]	eval's auc: 0.818757
[300]	eval's auc: 0.821218
[350]	eval's auc: 0.822865
[400]	eval's auc: 0.82454
[450]	eval's auc: 0.825585
[500]	eval's auc: 0.826543
[550]	eval's auc: 0.827338
[600]	eval's auc: 0.828133
[650]	eval's auc: 0.828672
[700]	eval's auc: 0.829277
[750]	eval's auc: 0.829879
[800]	eval's auc: 0.830219
[850]	eval's auc: 0.830579
[900]	eval's auc: 0.830967
[950]	eval's auc: 0.831361
Early stopping, best iteration is:
[955]	eval's auc: 0.831401
results:  0.8330088906717226 0.8298610780110138 0.6124787789834503


# Experiment 4
remove cat cols with 1 value + move rare occurences to one value + remove highly correlated features

In [10]:
def experiment4(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)

            X_train, cols_to_rm = dp.remove_single_val_cols(X_train, cat_cols_base)
            X_valid = X_valid.drop(columns=cols_to_rm)
            X_test = X_test.drop(columns=cols_to_rm)
            cat_cols_base = list(X_train.select_dtypes("category").columns)

            dp.rare_values_to_others(X_train, cat_cols_base)
            dp.rare_values_to_others(X_valid, cat_cols_base)
            dp.rare_values_to_others(X_test, cat_cols_base)

            num_cols_base = list(X_train.select_dtypes(include=np.number).columns)
            combs = list(combinations(num_cols_base, 2))
            corrs = dp.numeric_cols_correlation_check(X_train, combs)
            cols_to_rm = dp.get_corr_cols_to_rm(X_train, corrs)

            X_train = X_train.drop(columns=cols_to_rm)
            X_valid = X_valid.drop(columns=cols_to_rm)
            X_test = X_test.drop(columns=cols_to_rm)

            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)
            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))
        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "remove cat cols with 1 value + move rare occurences to one value + remove highly correlated features"
            },  
            "experiment4.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [11]:
bst, _, _, _ = experiment4(params)

bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L
lastapprcommoditycat_1041M P201_108_190
lastcancelreason_561M P91_110_150
lastrejectcommoditycat_161M P38_69_128
lastrejectcommodtypec_5251769M P34_45_168
lastrejectreason_759M P185_59_178
lastrejectreasonclient_4145040M P59_114_135
lastst_736L R
max_cancelreason_3545846M P91_110_150
max_rejectreason_755M P53_10_15
max_rejectreasonclient_4145042M P30_86_84
min_cancelreason_3545846M P91_110_150
min_rejectreasonclient_4145042M P59_114_135
first_cancelreason_3545846M P91_110_150
first_rejectreason_755M P185_59_178
first_rejectreasonclient_4145042M P59_114_135
last_cancelreason_3545846M P8_141_180
last_rejectreason_755M P19_25_34
last_rejectreasonclient_4145042M P64_121_167
mode_cancelreason_3545846M P91_110_150
mode_rej

C:\Users\wojtek\repos\kaggle_home_credit\data_proc.py:97: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr = pearsonr(data[comb[0]].fillna(0), data[comb[1]].fillna(0))[0]
C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.780997
[100]	eval's auc: 0.802648
[150]	eval's auc: 0.811764
[200]	eval's auc: 0.817194
[250]	eval's auc: 0.820324
[300]	eval's auc: 0.822675
[350]	eval's auc: 0.824401
[400]	eval's auc: 0.825983
[450]	eval's auc: 0.827236
[500]	eval's auc: 0.828387
[550]	eval's auc: 0.829125
[600]	eval's auc: 0.829839
[650]	eval's auc: 0.83061
[700]	eval's auc: 0.831134
[750]	eval's auc: 0.831789
[800]	eval's auc: 0.832283
[850]	eval's auc: 0.832594
[900]	eval's auc: 0.833191
Early stopping, best iteration is:
[910]	eval's auc: 0.833255
bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L
lastapprcommoditycat_1041M P3_103_80
lastcancelreason_561M P91_110_150
lastrejectcommoditycat_161M P38_69_128
lastrejec

C:\Users\wojtek\repos\kaggle_home_credit\data_proc.py:97: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr = pearsonr(data[comb[0]].fillna(0), data[comb[1]].fillna(0))[0]
C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.78261
[100]	eval's auc: 0.801306
[150]	eval's auc: 0.809816
[200]	eval's auc: 0.815206
[250]	eval's auc: 0.818365
[300]	eval's auc: 0.820655
[350]	eval's auc: 0.822265
[400]	eval's auc: 0.823525
[450]	eval's auc: 0.824682
[500]	eval's auc: 0.825606
[550]	eval's auc: 0.826307
[600]	eval's auc: 0.827327
[650]	eval's auc: 0.827957
Early stopping, best iteration is:
[659]	eval's auc: 0.828122
bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L
lastapprcommoditycat_1041M P201_108_190
lastcancelreason_561M P118_140_56
lastrejectcommoditycat_161M P38_69_128
lastrejectcommodtypec_5251769M P34_45_168
lastrejectreason_759M P185_59_178
lastrejectreasonclient_4145040M P203_151_99
lastst_736L R
max_can

...

Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.782575
[100]	eval's auc: 0.800829
[150]	eval's auc: 0.809762
[200]	eval's auc: 0.81481
[250]	eval's auc: 0.817996
[300]	eval's auc: 0.820443
[350]	eval's auc: 0.82207
[400]	eval's auc: 0.823652
[450]	eval's auc: 0.824661
[500]	eval's auc: 0.825646
[550]	eval's auc: 0.826604
[600]	eval's auc: 0.827308
[650]	eval's auc: 0.827945
[700]	eval's auc: 0.828539
[750]	eval's auc: 0.829285
[800]	eval's auc: 0.829671
[850]	eval's auc: 0.829975
[900]	eval's auc: 0.830384
[950]	eval's auc: 0.830793
[1000]	eval's auc: 0.831098
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.831098
bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L
lastapprcommoditycat_1041M P3_103_80
lastcancelreas

C:\Users\wojtek\repos\kaggle_home_credit\data_proc.py:97: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr = pearsonr(data[comb[0]].fillna(0), data[comb[1]].fillna(0))[0]
C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.779511
[100]	eval's auc: 0.79953
[150]	eval's auc: 0.80839
[200]	eval's auc: 0.813497
[250]	eval's auc: 0.816823
[300]	eval's auc: 0.819352
[350]	eval's auc: 0.821123
[400]	eval's auc: 0.822528
[450]	eval's auc: 0.823679
[500]	eval's auc: 0.824629
[550]	eval's auc: 0.825349
[600]	eval's auc: 0.826284
[650]	eval's auc: 0.826836
[700]	eval's auc: 0.827477
[750]	eval's auc: 0.82798
[800]	eval's auc: 0.828314
[850]	eval's auc: 0.828825
[900]	eval's auc: 0.829253
[950]	eval's auc: 0.829588
[1000]	eval's auc: 0.829847
Did not meet early stopping. Best iteration is:
[994]	eval's auc: 0.829868
bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L
lastapprcommoditycat_1041M P126_161_96
lastcancelreas

C:\Users\wojtek\repos\kaggle_home_credit\data_proc.py:97: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr = pearsonr(data[comb[0]].fillna(0), data[comb[1]].fillna(0))[0]
C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.784907
[100]	eval's auc: 0.805866
[150]	eval's auc: 0.81476
[200]	eval's auc: 0.819747
[250]	eval's auc: 0.823283
[300]	eval's auc: 0.825327
[350]	eval's auc: 0.827239
[400]	eval's auc: 0.828602
[450]	eval's auc: 0.829579
[500]	eval's auc: 0.830387
[550]	eval's auc: 0.831256
[600]	eval's auc: 0.831847
[650]	eval's auc: 0.832471
[700]	eval's auc: 0.832949
[750]	eval's auc: 0.833436
[800]	eval's auc: 0.833969
[850]	eval's auc: 0.834238
[900]	eval's auc: 0.834838
Early stopping, best iteration is:
[914]	eval's auc: 0.834867
results:  0.8314310799644898 0.8301864112372133 0.6271222868646427


# Experiment 5
fill several missing columns with a minimal value 

In [12]:
def experiment5(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            num_cols_base = list(X_train.select_dtypes(include=np.number).columns)
            
            dp.fill_min(X_train, num_cols_base)
            dp.fill_min(X_valid, num_cols_base)
            dp.fill_min(X_test, num_cols_base)
            
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)

            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))

        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "fill several missing columns with a minimal value"
            },  
            "experiment5.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [13]:
bst, _, _, _ = experiment5(params)

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.781956
[100]	eval's auc: 0.800679
[150]	eval's auc: 0.808867
[200]	eval's auc: 0.813824
[250]	eval's auc: 0.817074
[300]	eval's auc: 0.819264
[350]	eval's auc: 0.821532
[400]	eval's auc: 0.822877
[450]	eval's auc: 0.824091
[500]	eval's auc: 0.82507
[550]	eval's auc: 0.826067
[600]	eval's auc: 0.826861
[650]	eval's auc: 0.827471
[700]	eval's auc: 0.828144
[750]	eval's auc: 0.828794
[800]	eval's auc: 0.829282
[850]	eval's auc: 0.829805
[900]	eval's auc: 0.830118
[950]	eval's auc: 0.830486
Early stopping, best iteration is:
[950]	eval's auc: 0.830486


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.78001
[100]	eval's auc: 0.801223
[150]	eval's auc: 0.810746
[200]	eval's auc: 0.816415
[250]	eval's auc: 0.819769
[300]	eval's auc: 0.822228
[350]	eval's auc: 0.824095
[400]	eval's auc: 0.82544
[450]	eval's auc: 0.826671
[500]	eval's auc: 0.827588
[550]	eval's auc: 0.828444
[600]	eval's auc: 0.829227
[650]	eval's auc: 0.830055
[700]	eval's auc: 0.830641
[750]	eval's auc: 0.831212
[800]	eval's auc: 0.831857
[850]	eval's auc: 0.832305
Early stopping, best iteration is:
[855]	eval's auc: 0.832336


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.781239
[100]	eval's auc: 0.799573
[150]	eval's auc: 0.807745
[200]	eval's auc: 0.81295
[250]	eval's auc: 0.816454
[300]	eval's auc: 0.818764
[350]	eval's auc: 0.820723
[400]	eval's auc: 0.822375
[450]	eval's auc: 0.823554
[500]	eval's auc: 0.824797
[550]	eval's auc: 0.825546
[600]	eval's auc: 0.826288
[650]	eval's auc: 0.826971
[700]	eval's auc: 0.827355
[750]	eval's auc: 0.827747
[800]	eval's auc: 0.828088
[850]	eval's auc: 0.828424
[900]	eval's auc: 0.828721
[950]	eval's auc: 0.829109
Early stopping, best iteration is:
[957]	eval's auc: 0.829132


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.783652
[100]	eval's auc: 0.803625
[150]	eval's auc: 0.811864
[200]	eval's auc: 0.816593
[250]	eval's auc: 0.819584
[300]	eval's auc: 0.821885
[350]	eval's auc: 0.823666
[400]	eval's auc: 0.825272
[450]	eval's auc: 0.826541
[500]	eval's auc: 0.827481
[550]	eval's auc: 0.82838
[600]	eval's auc: 0.828924
[650]	eval's auc: 0.829492
[700]	eval's auc: 0.83006
[750]	eval's auc: 0.830529
[800]	eval's auc: 0.831
[850]	eval's auc: 0.831421
[900]	eval's auc: 0.831708
Early stopping, best iteration is:
[903]	eval's auc: 0.83172


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.782289
[100]	eval's auc: 0.801038
[150]	eval's auc: 0.809946
[200]	eval's auc: 0.81533
[250]	eval's auc: 0.818561
[300]	eval's auc: 0.820909
[350]	eval's auc: 0.822931
[400]	eval's auc: 0.824489
[450]	eval's auc: 0.82582
[500]	eval's auc: 0.826818
[550]	eval's auc: 0.827889
[600]	eval's auc: 0.828637
[650]	eval's auc: 0.829371
[700]	eval's auc: 0.830007
[750]	eval's auc: 0.830437
[800]	eval's auc: 0.831029
[850]	eval's auc: 0.831662
[900]	eval's auc: 0.831936
[950]	eval's auc: 0.832282
[1000]	eval's auc: 0.832612
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.832612
results:  0.831248684440402 0.8311437903848041 0.6162170293676899


# Experiment 6
fill missing columns with 0, where it seems to make sense

In [14]:
def experiment6(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            num_cols_base = list(X_train.select_dtypes(include=np.number).columns)
            
            dp.fill0(X_train, num_cols_base)
            dp.fill0(X_valid, num_cols_base)
            dp.fill0(X_test, num_cols_base)
            
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)

            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))

        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "fill several missing columns with a 0"
            },  
            "experiment6.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [15]:
bst, _, _, _ = experiment6(params)

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.780384
[100]	eval's auc: 0.800789
[150]	eval's auc: 0.810353
[200]	eval's auc: 0.815377
[250]	eval's auc: 0.818725
[300]	eval's auc: 0.82108
[350]	eval's auc: 0.822927
[400]	eval's auc: 0.824275
[450]	eval's auc: 0.825435
[500]	eval's auc: 0.826393
[550]	eval's auc: 0.827193
[600]	eval's auc: 0.827759
[650]	eval's auc: 0.828551
[700]	eval's auc: 0.829092
[750]	eval's auc: 0.829526
[800]	eval's auc: 0.830028
Early stopping, best iteration is:
[795]	eval's auc: 0.830052


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.781474
[100]	eval's auc: 0.802237
[150]	eval's auc: 0.810947
[200]	eval's auc: 0.815833
[250]	eval's auc: 0.819061
[300]	eval's auc: 0.821294
[350]	eval's auc: 0.823087
[400]	eval's auc: 0.824557
[450]	eval's auc: 0.825807
[500]	eval's auc: 0.826829
[550]	eval's auc: 0.827668
[600]	eval's auc: 0.828312
[650]	eval's auc: 0.828941
[700]	eval's auc: 0.829597
[750]	eval's auc: 0.830178
[800]	eval's auc: 0.830605
[850]	eval's auc: 0.831154
[900]	eval's auc: 0.831449
[950]	eval's auc: 0.831872
[1000]	eval's auc: 0.832142
Did not meet early stopping. Best iteration is:
[995]	eval's auc: 0.832155


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.784228
[100]	eval's auc: 0.802638
[150]	eval's auc: 0.811428
[200]	eval's auc: 0.815915
[250]	eval's auc: 0.818921
[300]	eval's auc: 0.820949
[350]	eval's auc: 0.822651
[400]	eval's auc: 0.823897
[450]	eval's auc: 0.825077
[500]	eval's auc: 0.826262
[550]	eval's auc: 0.826904
[600]	eval's auc: 0.827524
[650]	eval's auc: 0.828125
[700]	eval's auc: 0.828699
[750]	eval's auc: 0.829085
[800]	eval's auc: 0.829498
[850]	eval's auc: 0.830066
Early stopping, best iteration is:
[870]	eval's auc: 0.830175


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.781123
[100]	eval's auc: 0.801525
[150]	eval's auc: 0.810032
[200]	eval's auc: 0.814986
[250]	eval's auc: 0.818133
[300]	eval's auc: 0.820486
[350]	eval's auc: 0.822302
[400]	eval's auc: 0.823675
[450]	eval's auc: 0.824933
[500]	eval's auc: 0.825906
[550]	eval's auc: 0.826629
[600]	eval's auc: 0.827334
[650]	eval's auc: 0.828023
[700]	eval's auc: 0.828514
[750]	eval's auc: 0.829028
[800]	eval's auc: 0.829588
[850]	eval's auc: 0.830077
[900]	eval's auc: 0.830555
[950]	eval's auc: 0.83089
[1000]	eval's auc: 0.831106
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.831106


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.78233
[100]	eval's auc: 0.802194
[150]	eval's auc: 0.810979
[200]	eval's auc: 0.816099
[250]	eval's auc: 0.819617
[300]	eval's auc: 0.822234
[350]	eval's auc: 0.82425
[400]	eval's auc: 0.825688
[450]	eval's auc: 0.826935
[500]	eval's auc: 0.827786
[550]	eval's auc: 0.828726
[600]	eval's auc: 0.829476
[650]	eval's auc: 0.830193
[700]	eval's auc: 0.830805
[750]	eval's auc: 0.831472
[800]	eval's auc: 0.831981
[850]	eval's auc: 0.83233
[900]	eval's auc: 0.832604
[950]	eval's auc: 0.832926
[1000]	eval's auc: 0.833304
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.833304
results:  0.8313521480588018 0.8315462274728536 0.6236547350587023


# Experiment 7
add missing value indicators

In [16]:
def experiment7(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            num_cols_base = list(X_train.select_dtypes(include=np.number).columns)
            
            X_train = dp.indicate_missing_values(X_train)
            X_valid = dp.indicate_missing_values(X_valid)
            X_test = dp.indicate_missing_values(X_test)
            
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)

            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))

        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "add missing value indicators"
            },  
            "experiment7.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [17]:
bst, _, _, _ = experiment7(params)

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.783616
[100]	eval's auc: 0.803021
[150]	eval's auc: 0.811991
[200]	eval's auc: 0.817212
[250]	eval's auc: 0.820579
[300]	eval's auc: 0.822804
[350]	eval's auc: 0.824777
[400]	eval's auc: 0.826199
[450]	eval's auc: 0.827294
[500]	eval's auc: 0.828148
[550]	eval's auc: 0.828966
[600]	eval's auc: 0.829606
[650]	eval's auc: 0.830193
[700]	eval's auc: 0.830811
[750]	eval's auc: 0.831362
[800]	eval's auc: 0.831579
[850]	eval's auc: 0.832067
Early stopping, best iteration is:
[866]	eval's auc: 0.832162


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.780576
[100]	eval's auc: 0.799565
[150]	eval's auc: 0.809429
[200]	eval's auc: 0.814648
[250]	eval's auc: 0.818199
[300]	eval's auc: 0.820668
[350]	eval's auc: 0.822465
[400]	eval's auc: 0.824111
[450]	eval's auc: 0.825191
[500]	eval's auc: 0.826458
[550]	eval's auc: 0.827201
[600]	eval's auc: 0.828053
[650]	eval's auc: 0.828757
[700]	eval's auc: 0.829265
[750]	eval's auc: 0.829738
[800]	eval's auc: 0.830217
[850]	eval's auc: 0.830646
[900]	eval's auc: 0.830973
[950]	eval's auc: 0.831266
[1000]	eval's auc: 0.831421
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.831421


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.783703
[100]	eval's auc: 0.803196
[150]	eval's auc: 0.811185
[200]	eval's auc: 0.815856
[250]	eval's auc: 0.819007
[300]	eval's auc: 0.82163
[350]	eval's auc: 0.823415
[400]	eval's auc: 0.824491
[450]	eval's auc: 0.825668
[500]	eval's auc: 0.8266
[550]	eval's auc: 0.827633
[600]	eval's auc: 0.828433
[650]	eval's auc: 0.829021
[700]	eval's auc: 0.829555
[750]	eval's auc: 0.829887
[800]	eval's auc: 0.830203
[850]	eval's auc: 0.830713
[900]	eval's auc: 0.83096
[950]	eval's auc: 0.831234
Early stopping, best iteration is:
[978]	eval's auc: 0.831453


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.779133
[100]	eval's auc: 0.79971
[150]	eval's auc: 0.808838
[200]	eval's auc: 0.814139
[250]	eval's auc: 0.817766
[300]	eval's auc: 0.820024
[350]	eval's auc: 0.821887
[400]	eval's auc: 0.823338
[450]	eval's auc: 0.824805
[500]	eval's auc: 0.825675
[550]	eval's auc: 0.826441
[600]	eval's auc: 0.827248
[650]	eval's auc: 0.82795
[700]	eval's auc: 0.828502
[750]	eval's auc: 0.828966
[800]	eval's auc: 0.829375
[850]	eval's auc: 0.829876
[900]	eval's auc: 0.830299
[950]	eval's auc: 0.830632
[1000]	eval's auc: 0.830919
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.830919


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.783634
[100]	eval's auc: 0.802966
[150]	eval's auc: 0.81228
[200]	eval's auc: 0.817182
[250]	eval's auc: 0.820798
[300]	eval's auc: 0.822876
[350]	eval's auc: 0.824672
[400]	eval's auc: 0.82615
[450]	eval's auc: 0.827206
[500]	eval's auc: 0.828233
[550]	eval's auc: 0.828975
[600]	eval's auc: 0.829624
[650]	eval's auc: 0.830268
[700]	eval's auc: 0.830837
[750]	eval's auc: 0.831273
[800]	eval's auc: 0.831751
[850]	eval's auc: 0.832176
[900]	eval's auc: 0.83265
Early stopping, best iteration is:
[934]	eval's auc: 0.832806
results:  0.8317407169630456 0.8324183154241034 0.6233953012286602


# Experiment 8
like experiment 5 but with missing value indicators

In [18]:
def experiment8(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            num_cols_base = list(X_train.select_dtypes(include=np.number).columns)

            X_train = dp.indicate_missing_values(X_train)
            X_valid = dp.indicate_missing_values(X_valid)
            X_test = dp.indicate_missing_values(X_test)

            dp.fill_min(X_train, num_cols_base)
            dp.fill_min(X_valid, num_cols_base)
            dp.fill_min(X_test, num_cols_base)
            
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)

            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))

        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "fill several missing columns with a minimal value + missing value indicators"
            },  
            "experiment8.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [19]:
bst, _, _, _ = experiment8(params)

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.779815
[100]	eval's auc: 0.799628
[150]	eval's auc: 0.808334
[200]	eval's auc: 0.812996
[250]	eval's auc: 0.816062
[300]	eval's auc: 0.818339
[350]	eval's auc: 0.820215
[400]	eval's auc: 0.821571
[450]	eval's auc: 0.822848
[500]	eval's auc: 0.823766
[550]	eval's auc: 0.824573
[600]	eval's auc: 0.82542
[650]	eval's auc: 0.825952
[700]	eval's auc: 0.826443
[750]	eval's auc: 0.826954
[800]	eval's auc: 0.827548
[850]	eval's auc: 0.827809
[900]	eval's auc: 0.828106
[950]	eval's auc: 0.82844
[1000]	eval's auc: 0.82879
Did not meet early stopping. Best iteration is:
[998]	eval's auc: 0.828803


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.781158
[100]	eval's auc: 0.801651
[150]	eval's auc: 0.810817
[200]	eval's auc: 0.815694
[250]	eval's auc: 0.818891
[300]	eval's auc: 0.82134
[350]	eval's auc: 0.823193
[400]	eval's auc: 0.824504
[450]	eval's auc: 0.82564
[500]	eval's auc: 0.826548
[550]	eval's auc: 0.827522
[600]	eval's auc: 0.82829
[650]	eval's auc: 0.828887
[700]	eval's auc: 0.829424
[750]	eval's auc: 0.829899
[800]	eval's auc: 0.830435
[850]	eval's auc: 0.830844
[900]	eval's auc: 0.831035
[950]	eval's auc: 0.831392
[1000]	eval's auc: 0.831631
Did not meet early stopping. Best iteration is:
[999]	eval's auc: 0.831641


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.785068
[100]	eval's auc: 0.804508
[150]	eval's auc: 0.812682
[200]	eval's auc: 0.817296
[250]	eval's auc: 0.820846
[300]	eval's auc: 0.823089
[350]	eval's auc: 0.825053
[400]	eval's auc: 0.826479
[450]	eval's auc: 0.827622
[500]	eval's auc: 0.828645
[550]	eval's auc: 0.829436
[600]	eval's auc: 0.829988
[650]	eval's auc: 0.830654
[700]	eval's auc: 0.831367
[750]	eval's auc: 0.831769
[800]	eval's auc: 0.832244
[850]	eval's auc: 0.832681
[900]	eval's auc: 0.833193
[950]	eval's auc: 0.833632
[1000]	eval's auc: 0.83391
Did not meet early stopping. Best iteration is:
[995]	eval's auc: 0.833922


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.780549
[100]	eval's auc: 0.800281
[150]	eval's auc: 0.809252
[200]	eval's auc: 0.814346
[250]	eval's auc: 0.817687
[300]	eval's auc: 0.820063
[350]	eval's auc: 0.821958
[400]	eval's auc: 0.823231
[450]	eval's auc: 0.824343
[500]	eval's auc: 0.825353
[550]	eval's auc: 0.826363
[600]	eval's auc: 0.826941
[650]	eval's auc: 0.827601
[700]	eval's auc: 0.828099
[750]	eval's auc: 0.82857
[800]	eval's auc: 0.829044
[850]	eval's auc: 0.829302
[900]	eval's auc: 0.829776
[950]	eval's auc: 0.830162
Early stopping, best iteration is:
[960]	eval's auc: 0.830206


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.783187
[100]	eval's auc: 0.803779
[150]	eval's auc: 0.812028
[200]	eval's auc: 0.817698
[250]	eval's auc: 0.820898
[300]	eval's auc: 0.82324
[350]	eval's auc: 0.824983
[400]	eval's auc: 0.826226
[450]	eval's auc: 0.82714
[500]	eval's auc: 0.828197
[550]	eval's auc: 0.828944
[600]	eval's auc: 0.829522
[650]	eval's auc: 0.83039
[700]	eval's auc: 0.831053
[750]	eval's auc: 0.831455
[800]	eval's auc: 0.832058
[850]	eval's auc: 0.832346
[900]	eval's auc: 0.832676
[950]	eval's auc: 0.833153
[1000]	eval's auc: 0.83339
Did not meet early stopping. Best iteration is:
[999]	eval's auc: 0.833396
results:  0.831584087511253 0.831734997385888 0.6242364454355912


# Experiment 9
like experiment 6 but with missing value indicators

In [21]:
def experiment9(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            num_cols_base = list(X_train.select_dtypes(include=np.number).columns)

            X_train = dp.indicate_missing_values(X_train)
            X_valid = dp.indicate_missing_values(X_valid)
            X_test = dp.indicate_missing_values(X_test)
            dp.fill0(X_train, num_cols_base)
            dp.fill0(X_valid, num_cols_base)
            dp.fill0(X_test, num_cols_base)
            
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)

            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))

        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "fill several missing columns with a 0 + missing value indicators"
            },  
            "experiment9.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [22]:
bst, _, _, _ = experiment6(params)

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.782758
[100]	eval's auc: 0.802501
[150]	eval's auc: 0.81145
[200]	eval's auc: 0.81675
[250]	eval's auc: 0.820255
[300]	eval's auc: 0.822498
[350]	eval's auc: 0.824493
[400]	eval's auc: 0.826016
[450]	eval's auc: 0.827104
[500]	eval's auc: 0.827992
[550]	eval's auc: 0.829054
[600]	eval's auc: 0.829689
[650]	eval's auc: 0.830387
[700]	eval's auc: 0.830853
[750]	eval's auc: 0.831223
[800]	eval's auc: 0.831749
[850]	eval's auc: 0.832159
[900]	eval's auc: 0.832432
[950]	eval's auc: 0.832769
Early stopping, best iteration is:
[956]	eval's auc: 0.832818


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.781702
[100]	eval's auc: 0.801027
[150]	eval's auc: 0.80988
[200]	eval's auc: 0.814938
[250]	eval's auc: 0.818009
[300]	eval's auc: 0.820582
[350]	eval's auc: 0.822457
[400]	eval's auc: 0.823805
[450]	eval's auc: 0.825118
[500]	eval's auc: 0.826144
[550]	eval's auc: 0.827009
[600]	eval's auc: 0.827685
[650]	eval's auc: 0.828537
[700]	eval's auc: 0.829014
[750]	eval's auc: 0.829455
[800]	eval's auc: 0.829891
[850]	eval's auc: 0.830347
[900]	eval's auc: 0.830598
Early stopping, best iteration is:
[892]	eval's auc: 0.830622


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.78247
[100]	eval's auc: 0.802604
[150]	eval's auc: 0.810915
[200]	eval's auc: 0.816186
[250]	eval's auc: 0.819114
[300]	eval's auc: 0.821466
[350]	eval's auc: 0.823428
[400]	eval's auc: 0.824755
[450]	eval's auc: 0.825861
[500]	eval's auc: 0.826833
[550]	eval's auc: 0.827642
[600]	eval's auc: 0.828475
[650]	eval's auc: 0.829185
[700]	eval's auc: 0.829666
[750]	eval's auc: 0.83034
[800]	eval's auc: 0.830752
[850]	eval's auc: 0.831199
[900]	eval's auc: 0.831719
[950]	eval's auc: 0.832082
[1000]	eval's auc: 0.832419
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.832419


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.787045
[100]	eval's auc: 0.806476
[150]	eval's auc: 0.81508
[200]	eval's auc: 0.820036
[250]	eval's auc: 0.823211
[300]	eval's auc: 0.825265
[350]	eval's auc: 0.827196
[400]	eval's auc: 0.828641
[450]	eval's auc: 0.829882
[500]	eval's auc: 0.830699
[550]	eval's auc: 0.831566
[600]	eval's auc: 0.83231
[650]	eval's auc: 0.832936
[700]	eval's auc: 0.833391
[750]	eval's auc: 0.833931
[800]	eval's auc: 0.834411
[850]	eval's auc: 0.834797
[900]	eval's auc: 0.8352
[950]	eval's auc: 0.83554
[1000]	eval's auc: 0.835816
Did not meet early stopping. Best iteration is:
[998]	eval's auc: 0.83582


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.780988
[100]	eval's auc: 0.801046
[150]	eval's auc: 0.809847
[200]	eval's auc: 0.814926
[250]	eval's auc: 0.818168
[300]	eval's auc: 0.82037
[350]	eval's auc: 0.82201
[400]	eval's auc: 0.823417
[450]	eval's auc: 0.824485
[500]	eval's auc: 0.825489
[550]	eval's auc: 0.826377
[600]	eval's auc: 0.827071
[650]	eval's auc: 0.827659
[700]	eval's auc: 0.828141
[750]	eval's auc: 0.828719
[800]	eval's auc: 0.829147
[850]	eval's auc: 0.829401
[900]	eval's auc: 0.829717
Early stopping, best iteration is:
[926]	eval's auc: 0.829935
results:  0.8323125156114418 0.8331996551436976 0.6321531123185136


# Experiment 10
Try with some custom NA fills from kaggle noteook

In [23]:
def experiment10(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base_filled.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)
            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))
        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "Try with some custom NA fills from kaggle noteook"
            },  
            "experiment10.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [26]:
bst, _, _, _ = experiment10(params)

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.784097
[100]	eval's auc: 0.803847
[150]	eval's auc: 0.812561
[200]	eval's auc: 0.817582
[250]	eval's auc: 0.820877
[300]	eval's auc: 0.823218
[350]	eval's auc: 0.824744
[400]	eval's auc: 0.826195
[450]	eval's auc: 0.8273
[500]	eval's auc: 0.828082
[550]	eval's auc: 0.828975
[600]	eval's auc: 0.829664
[650]	eval's auc: 0.830332
[700]	eval's auc: 0.830954
[750]	eval's auc: 0.8313
Early stopping, best iteration is:
[742]	eval's auc: 0.831314


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.778424
[100]	eval's auc: 0.798439
[150]	eval's auc: 0.80726
[200]	eval's auc: 0.812608
[250]	eval's auc: 0.816047
[300]	eval's auc: 0.818619
[350]	eval's auc: 0.820515
[400]	eval's auc: 0.82174
[450]	eval's auc: 0.822991
[500]	eval's auc: 0.824041
[550]	eval's auc: 0.824927
[600]	eval's auc: 0.825685
[650]	eval's auc: 0.826238
[700]	eval's auc: 0.826806
[750]	eval's auc: 0.827548
[800]	eval's auc: 0.828036
[850]	eval's auc: 0.828304
[900]	eval's auc: 0.828737
[950]	eval's auc: 0.829031
[1000]	eval's auc: 0.829329
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.829329


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.783339
[100]	eval's auc: 0.803774
[150]	eval's auc: 0.81177
[200]	eval's auc: 0.816696
[250]	eval's auc: 0.81985
[300]	eval's auc: 0.822041
[350]	eval's auc: 0.823924
[400]	eval's auc: 0.825078
[450]	eval's auc: 0.8262
[500]	eval's auc: 0.827086
[550]	eval's auc: 0.827896
[600]	eval's auc: 0.828572
[650]	eval's auc: 0.829039
[700]	eval's auc: 0.829573
[750]	eval's auc: 0.829922
[800]	eval's auc: 0.830258
[850]	eval's auc: 0.830739
[900]	eval's auc: 0.831285
[950]	eval's auc: 0.831553
Early stopping, best iteration is:
[943]	eval's auc: 0.831594


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.781659
[100]	eval's auc: 0.801555
[150]	eval's auc: 0.810168
[200]	eval's auc: 0.815537
[250]	eval's auc: 0.819132
[300]	eval's auc: 0.821669
[350]	eval's auc: 0.823662
[400]	eval's auc: 0.825277
[450]	eval's auc: 0.826461
[500]	eval's auc: 0.82762
[550]	eval's auc: 0.828446
[600]	eval's auc: 0.829222
[650]	eval's auc: 0.830045
[700]	eval's auc: 0.830521
[750]	eval's auc: 0.830994
[800]	eval's auc: 0.831532
[850]	eval's auc: 0.832225
[900]	eval's auc: 0.832615
[950]	eval's auc: 0.833044
[1000]	eval's auc: 0.833163
Did not meet early stopping. Best iteration is:
[993]	eval's auc: 0.833219


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.779888
[100]	eval's auc: 0.798813
[150]	eval's auc: 0.808278
[200]	eval's auc: 0.813382
[250]	eval's auc: 0.81641
[300]	eval's auc: 0.818869
[350]	eval's auc: 0.820694
[400]	eval's auc: 0.822215
[450]	eval's auc: 0.823314
[500]	eval's auc: 0.824221
[550]	eval's auc: 0.825081
[600]	eval's auc: 0.825846
[650]	eval's auc: 0.826534
[700]	eval's auc: 0.827155
[750]	eval's auc: 0.827675
[800]	eval's auc: 0.8282
[850]	eval's auc: 0.828637
[900]	eval's auc: 0.828982
Early stopping, best iteration is:
[904]	eval's auc: 0.828998
results:  0.8308693954733652 0.8306688740103102 0.6149985307017999


# Experiment 11
add missing value indicators and impute missing values with knn

In [5]:
corr_cols_to_rm = ['first_empl_employedfrom_271D',
 'last_childnum_21L',
 'first_pmtamount_36A',
 'maxdpdlast24m_143P',
 'mean_dtlastpmt_581D',
 'avgdpdtolclosure24_3658938P',
 'numinstpaidearly5dobd_4499205L',
 'first_num_group1_1_5',
 'numberofqueries_373L',
 'max_num_group1_1_3',
 'mean_dtlastpmtallstes_3545839D',
 'firstdatedue_489D',
 'pmtaverage_4527227A',
 'min_employedfrom_700D',
 'first_amount_4917619A',
 'credamount_770A',
 'pctinstlsallpaidlat10d_839L',
 'mean_openingdate_857D',
 'last_persontype_1072L',
 'last_actualdpd_943P',
 'numinstpaidearly_338L',
 'max_maxdpdtolerance_577P',
 'first_tenor_203L',
 'first_creationdate_885D',
 'numinstlsallpaid_934L',
 'max_num_group1_1_7',
 'min_tenor_203L',
 'mean_credacc_actualbalance_314A',
 'days180_256L',
 'first_dateactivated_425D',
 'last_tenor_203L',
 'mean_actualdpd_943P',
 'mean_empl_employedfrom_271D',
 'mean_mainoccupationinc_437A',
 'last_dtlastpmt_581D',
 'numinstlswithoutdpd_562L',
 'min_recorddate_4527225D',
 'currdebt_22A',
 'mean_openingdate_313D',
 'disbursedcredamount_1113A',
 'min_approvaldate_319D',
 'mean_employedfrom_700D',
 'min_openingdate_857D',
 'maxdbddpdtollast6m_4187119P',
 'first_amount_4527230A',
 'last_deductiondate_4917603D',
 'first_num_group1_1_4',
 'maxdpdlast6m_474P',
 'first_deductiondate_4917603D',
 'max_persontype_1072L',
 'avgdbddpdlast24m_3658932P',
 'first_firstnonzeroinstldate_307D',
 'mean_mainoccupationinc_384A',
 'numinstregularpaid_973L',
 'actualdpdtolerance_344P',
 'min_firstnonzeroinstldate_307D',
 'avgoutstandbalancel6m_4187114A',
 'min_credacc_actualbalance_314A',
 'max_persontype_792L',
 'avgdbddpdlast3m_4187120P',
 'max_creationdate_885D',
 'last_amount_4527230A',
 'first_amount_416A',
 'sumoutstandtotalest_4493215A',
 'maxdpdfrom6mto36m_3546853P',
 'maxdpdlast9m_1059P',
 'mean_recorddate_4527225D',
 'days90_310L',
 'last_maxdpdtolerance_577P',
 'lastactivateddate_801D',
 'interestrate_311L',
 'mean_maxdpdtolerance_577P',
 'last_num_group1_1_5',
 'last_creationdate_885D',
 'last_num_group1_1_3',
 'mean_birth_259D',
 'last_num_group1_1_8',
 'mean_annuity_853A',
 'min_credacc_maxhisbal_375A',
 'last_openingdate_857D',
 'first_openingdate_857D',
 'mean_amount_4527230A',
 'min_credacc_minhisbal_90A',
 'first_birth_259D',
 'applicationscnt_629L',
 'numinstregularpaidest_4493210L',
 'datelastunpaid_3546854D',
 'max_firstnonzeroinstldate_307D',
 'max_dateactivated_425D',
 'mean_dateactivated_425D',
 'first_dtlastpmt_581D',
 'last_num_group1',
 'clientscnt3m_3712950L',
 'last_processingdate_168D',
 'min_mainoccupationinc_384A',
 'max_currdebt_94A',
 'mean_credamount_590A',
 'min_openingdate_313D',
 'min_empl_employedfrom_271D',
 'max_num_group1_1_8',
 'mean_amount_4917619A',
 'maxdpdtolerance_374P',
 'last_pmtamount_36A',
 'first_actualdpd_943P',
 'mean_processingdate_168D',
 'min_outstandingdebt_522A',
 'numincomingpmts_3546848L',
 'numinstpaid_4499208L',
 'first_mainoccupationinc_437A',
 'mean_firstnonzeroinstldate_307D',
 'max_approvaldate_319D',
 'mean_credacc_minhisbal_90A',
 'min_dateactivated_425D',
 'datefirstoffer_1144D',
 'first_num_group1_1_3',
 'numinstpaidearlyest_4493214L',
 'pmtaverage_3A',
 'mean_credacc_maxhisbal_375A',
 'max_openingdate_857D',
 'mean_approvaldate_319D',
 'first_mainoccupationinc_384A',
 'numinstpaidearly3d_3546850L',
 'max_num_group1_1_5',
 'numinsttopaygr_769L',
 'maxoutstandbalancel12m_4187113A',
 'numinstunpaidmaxest_4493212L',
 'last_approvaldate_319D',
 'max_pmtamount_36A',
 'days360_512L',
 'pctinstlsallpaidlate4d_3546849L',
 'sumoutstandtotal_3546847A',
 'clientscnt_1022L',
 'mindbddpdlast24m_3658935P',
 'last_num_group1_1_7',
 'first_processingdate_168D',
 'last_outstandingdebt_522A',
 'last_employedfrom_700D',
 'pctinstlsallpaidlate6d_3546844L',
 'first_openingdate_313D',
 'first_outstandingdebt_522A',
 'last_amount_416A',
 'pmtcount_4527229L',
 'maxdbddpdlast1m_3658939P',
 'mean_deductiondate_4917603D',
 'last_num_group1_1_9',
 'last_mainoccupationinc_437A',
 'mean_creationdate_885D',
 'monthsannuity_845L',
 'last_amount_4917619A',
 'last_firstnonzeroinstldate_307D',
 'maxdbddpdtollast12m_3658940P',
 'last_dateactivated_425D',
 'max_outstandingdebt_522A',
 'last_recorddate_4527225D',
 'mindbdtollast24m_4525191P',
 'clientscnt6m_3712949L',
 'numinstmatpaidtearly2d_4499204L',
 'first_num_group1_1_8',
 'last_num_group1_1_4',
 'first_recorddate_4527225D',
 'mean_outstandingdebt_522A',
 'max_tenor_203L',
 'numinstunpaidmax_3546851L',
 'mean_pmtamount_36A',
 'min_birth_259D',
 'sellerplacescnt_216L',
 'mean_revolvingaccount_394A',
 'last_persontype_792L',
 'mean_amount_416A',
 'last_openingdate_313D',
 'min_dtlastpmt_581D',
 'min_revolvingaccount_394A']

In [6]:
cat_to_knn = ["max_sex_738L", "min_cancelreason_3545846M" , "education_1103M", "max_incometype_1044T", "education_88M", "maritalst_385M"]

In [24]:
X_train[cat_to_knn].head()

,max_sex_738L,min_cancelreason_3545846M,education_1103M,max_incometype_1044T,education_88M,maritalst_385M
0,F,NaN,NaN,SALARIED_GOVT,NaN,NaN
1,F,a55475b1,NaN,EMPLOYED,NaN,NaN
2,F,P94_109_143,NaN,EMPLOYED,NaN,NaN
3,F,P24_27_36,NaN,EMPLOYED,NaN,NaN
4,F,P85_114_140,NaN,PRIVATE_SECTOR_EMPLOYEE,NaN,NaN


In [9]:
def experiment11(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            num_cols_base = list(X_train.select_dtypes(include=np.number).columns)

            X_train, cols_to_rm = dp.remove_single_val_cols(X_train, cat_cols_base)
            X_valid = X_valid.drop(columns=cols_to_rm)
            X_test = X_test.drop(columns=cols_to_rm)
            cat_cols_base = list(X_train.select_dtypes("category").columns)

            dp.rare_values_to_others(X_train, cat_cols_base)
            dp.rare_values_to_others(X_valid, cat_cols_base)
            dp.rare_values_to_others(X_test, cat_cols_base)

            X_valid = X_valid.drop(columns=corr_cols_to_rm)
            X_test = X_test.drop(columns=corr_cols_to_rm)
            X_train = X_train.drop(columns=corr_cols_to_rm)

            X_train = dp.indicate_missing_values(X_train)
            X_valid = dp.indicate_missing_values(X_valid)
            X_test = dp.indicate_missing_values(X_test)

            dp.fill0(X_train, num_cols_base)
            dp.fill0(X_valid, num_cols_base)
            dp.fill0(X_test, num_cols_base)

            dp.fill_min(X_train, num_cols_base)
            dp.fill_min(X_valid, num_cols_base)
            dp.fill_min(X_test, num_cols_base)

            #X_train = dp.dummify_categorical(X_train, cat_to_knn)
            print(X_train.shape)
            #X_valid = dp.dummify_categorical(X_valid, cat_to_knn)
            print(X_valid.shape)
            #X_test = dp.dummify_categorical(X_test, cat_to_knn)
            print(X_test.shape)

            num_cols_base = list(X_train.select_dtypes(include=np.number).columns)
            X_train[num_cols_base], imputer = dp.fill_lr(X_train[num_cols_base])
            print("X_train filled!") 
            X_valid[num_cols_base] = imputer.transform(X_valid[num_cols_base])
            print("X_valid filled!") 
            X_test[num_cols_base] = imputer.transform(X_test[num_cols_base])
            print("X_test filled!") 
            
            train_data = lgb.Dataset(X_train, label=y_train)
            valid_data = lgb.Dataset(X_valid, label=y_valid)
            test_data = lgb.Dataset(X_test, label=y_test)

            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))

        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "add missing value indicators and impute missing values with knn"
            },  
            "experiment11.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [11]:
bst, _, _, _ = experiment11(params)

bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L
education_88M c8e1a1d0
maritalst_893M ecd83604
lastapprcommoditycat_1041M P143_21_170
lastcancelreason_561M P32_86_86
lastrejectcommoditycat_161M P110_120_162
lastrejectcommodtypec_5251769M P135_89_156
lastrejectreason_759M P53_10_15
lastrejectreasonclient_4145040M P59_114_135
lastst_736L H
max_cancelreason_3545846M P53_10_15
max_rejectreason_755M P53_10_15
max_rejectreasonclient_4145042M P84_14_61
min_cancelreason_3545846M P32_86_86
min_rejectreason_755M P53_10_15
min_rejectreasonclient_4145042M P59_114_135
first_cancelreason_3545846M P32_86_86
first_rejectreason_755M P53_10_15
first_rejectreasonclient_4145042M P53_10_15
last_cancelreason_3545846M P53_10_15
last_rejectreason_755M P53_10_15
last_rejectreasonclient_41

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


X_train filled!
X_valid filled!
X_test filled!


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.777479
[100]	eval's auc: 0.79805
[150]	eval's auc: 0.806702
[200]	eval's auc: 0.81193
[250]	eval's auc: 0.814935
[300]	eval's auc: 0.817285
[350]	eval's auc: 0.819105
[400]	eval's auc: 0.820537
[450]	eval's auc: 0.821638
[500]	eval's auc: 0.8226
[550]	eval's auc: 0.823234
[600]	eval's auc: 0.82376
[650]	eval's auc: 0.82417
[700]	eval's auc: 0.824724
[750]	eval's auc: 0.825165
[800]	eval's auc: 0.825742
[850]	eval's auc: 0.825946
[900]	eval's auc: 0.826169
[950]	eval's auc: 0.826448
Early stopping, best iteration is:
[988]	eval's auc: 0.826756
bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L
education_88M c8e1a1d0
maritalst_893M ecd83604
lastapprcommoditycat_1041M P143_21_170
lastcancelr

...

Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.776312
[100]	eval's auc: 0.797421
[150]	eval's auc: 0.806561
[200]	eval's auc: 0.811799
[250]	eval's auc: 0.815414
[300]	eval's auc: 0.817676
[350]	eval's auc: 0.819689
[400]	eval's auc: 0.82112
[450]	eval's auc: 0.822567
[500]	eval's auc: 0.823548
[550]	eval's auc: 0.824433
[600]	eval's auc: 0.824944
[650]	eval's auc: 0.825666
[700]	eval's auc: 0.826164
[750]	eval's auc: 0.826608
[800]	eval's auc: 0.827069
[850]	eval's auc: 0.827482
Early stopping, best iteration is:
[845]	eval's auc: 0.827504
bankacctype_710L
isdebitcard_729L
paytype1st_925L
paytype_783L
typesuite_864L
min_isbidproduct_390L
max_contaddr_matchlist_1032L
max_remitter_829L
min_contaddr_matchlist_1032L
min_remitter_829L
first_contaddr_matchlist_1032L
last_contaddr_matchlist_1032L
last_remitter_829L
education_88M c8e1a1d0
maritalst_893M ecd83604
lastapprcommoditycat_1041M P143_21_170
lastcancelreason_561M P8_141_180
lastrejectcommoditycat_161M

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


X_train filled!
X_valid filled!
X_test filled!


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.779141
[100]	eval's auc: 0.800214
[150]	eval's auc: 0.808359
[200]	eval's auc: 0.813238
[250]	eval's auc: 0.816109
[300]	eval's auc: 0.818337
[350]	eval's auc: 0.820024
[400]	eval's auc: 0.821443
[450]	eval's auc: 0.822661
[500]	eval's auc: 0.823537
[550]	eval's auc: 0.824314
[600]	eval's auc: 0.825129
[650]	eval's auc: 0.82588
[700]	eval's auc: 0.82637
[750]	eval's auc: 0.826745
[800]	eval's auc: 0.827378
[850]	eval's auc: 0.827647
[900]	eval's auc: 0.828178
[950]	eval's auc: 0.828419
Early stopping, best iteration is:
[982]	eval's auc: 0.828653
results:  0.8285971079198523 0.8269855812452237 0.6156987805730794


# Experiment 12
try increasing/decreasing depth

In [16]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 2,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

In [17]:
def experiment12(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)
            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))
        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "try increasing/decreasing depth"
            },  
            "experiment12.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [18]:
bst, _, _, _ = experiment12(params)

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.766633
[100]	eval's auc: 0.787706
[150]	eval's auc: 0.798246
[200]	eval's auc: 0.804227
[250]	eval's auc: 0.807876
[300]	eval's auc: 0.810681
[350]	eval's auc: 0.81312
[400]	eval's auc: 0.814673
[450]	eval's auc: 0.816157
[500]	eval's auc: 0.817562
[550]	eval's auc: 0.818613
[600]	eval's auc: 0.81969
[650]	eval's auc: 0.820509
[700]	eval's auc: 0.82134
[750]	eval's auc: 0.822034
[800]	eval's auc: 0.822506
[850]	eval's auc: 0.822996
[900]	eval's auc: 0.823579
[950]	eval's auc: 0.823972
[1000]	eval's auc: 0.824287
Did not meet early stopping. Best iteration is:
[998]	eval's auc: 0.824291


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.766699
[100]	eval's auc: 0.789171
[150]	eval's auc: 0.800224
[200]	eval's auc: 0.80661
[250]	eval's auc: 0.810595
[300]	eval's auc: 0.813633
[350]	eval's auc: 0.815982
[400]	eval's auc: 0.817992
[450]	eval's auc: 0.819449
[500]	eval's auc: 0.820832
[550]	eval's auc: 0.821842
[600]	eval's auc: 0.822778
[650]	eval's auc: 0.823683
[700]	eval's auc: 0.824421
[750]	eval's auc: 0.825266
[800]	eval's auc: 0.825873
[850]	eval's auc: 0.826446
[900]	eval's auc: 0.827085
[950]	eval's auc: 0.82751
[1000]	eval's auc: 0.827924
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.827924


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.762838
[100]	eval's auc: 0.788393
[150]	eval's auc: 0.800035
[200]	eval's auc: 0.806307
[250]	eval's auc: 0.810358
[300]	eval's auc: 0.813194
[350]	eval's auc: 0.815668
[400]	eval's auc: 0.817518
[450]	eval's auc: 0.818959
[500]	eval's auc: 0.820262
[550]	eval's auc: 0.821203
[600]	eval's auc: 0.82224
[650]	eval's auc: 0.823078
[700]	eval's auc: 0.823832
[750]	eval's auc: 0.824465
[800]	eval's auc: 0.825118
[850]	eval's auc: 0.825675
[900]	eval's auc: 0.826172
[950]	eval's auc: 0.82658
[1000]	eval's auc: 0.827137
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.827137


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.76377
[100]	eval's auc: 0.787518
[150]	eval's auc: 0.798674
[200]	eval's auc: 0.804419
[250]	eval's auc: 0.808689
[300]	eval's auc: 0.811808
[350]	eval's auc: 0.81406
[400]	eval's auc: 0.816078
[450]	eval's auc: 0.817537
[500]	eval's auc: 0.818738
[550]	eval's auc: 0.82
[600]	eval's auc: 0.820893
[650]	eval's auc: 0.821805
[700]	eval's auc: 0.822527
[750]	eval's auc: 0.82314
[800]	eval's auc: 0.823668
[850]	eval's auc: 0.824358
[900]	eval's auc: 0.824749
[950]	eval's auc: 0.82523
[1000]	eval's auc: 0.825654
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.825654


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.767035
[100]	eval's auc: 0.790605
[150]	eval's auc: 0.801834
[200]	eval's auc: 0.808308
[250]	eval's auc: 0.812923
[300]	eval's auc: 0.816247
[350]	eval's auc: 0.818643
[400]	eval's auc: 0.82041
[450]	eval's auc: 0.821976
[500]	eval's auc: 0.823402
[550]	eval's auc: 0.82445
[600]	eval's auc: 0.825371
[650]	eval's auc: 0.826126
[700]	eval's auc: 0.827012
[750]	eval's auc: 0.827646
[800]	eval's auc: 0.828269
[850]	eval's auc: 0.828882
[900]	eval's auc: 0.829387
[950]	eval's auc: 0.829885
[1000]	eval's auc: 0.830215
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.830215
results:  0.8270434672963318 0.8270543488673511 0.6165354579772907


# hyperparam tuning

In [5]:
(base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
cat_cols_base = list(X_train.select_dtypes("category").columns)
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base, free_raw_data=False)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base, free_raw_data=False)
test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base, free_raw_data=False)

In [6]:
EXPERIMENT_NAME = "credit-score-tuning"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/775714010158325685', creation_time=1713352331682, experiment_id='775714010158325685', last_update_time=1713352331682, lifecycle_stage='active', name='credit-score-tuning', tags={}>

In [7]:
def hyperopt_search(train_data, valid_data, X_test, y_test, base_test):
    eval_result = {}
    def objective(params):
        with mlflow.start_run():
            mlflow.set_tag("model", "lgb")
            mlflow.log_params(params)
            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)]
            )
            auc = eval_result['eval']['auc'][-1]
            mlflow.log_metric("auc-eval", auc)
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            mlflow.log_metric("auc-test", roc_auc_score(y_test.values, ypred))
            mlflow.log_metric("gini-stability-test", gini_stability(base_test))

        return {"loss": -auc, "status": STATUS_OK}

    search_space = {
        'feature_pre_filter': hp.choice('feature_pre_filter', [False]),
        'objective': hp.choice('objective', ['binary']),
        'metric': hp.choice('metric', ['auc']),
        'feature_fraction': hp.uniform('feature_fraction', 0.5, 0.9),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 0.9),
        'bagging_freq': hp.choice('bagging_freq', [5]),
        'n_estimators': hp.choice('n_estimators', [1000]),
        'verbose': hp.choice('verbose', [-1]),
        'learning_rate': hp.choice('learning_rate', [0.05, 0.1]),
        'num_leaves': hp.randint('num_leaves', 6, 50), 
        'min_child_samples': hp.randint('min_child_samples', 100, 500), 
        'min_child_weight': hp.loguniform('min_child_weight', -5, 4),
        'subsample': hp.uniform('subsample', 0.2, 1), 
        'colsample_bytree': hp.uniform('colsample_bytree', 0.4, 1),
        'reg_alpha': hp.choice('reg_alpha', [0, 1e-1, 1, 2, 5, 7, 10, 100]),
        'reg_lambda': hp.choice('reg_lambda', [0, 1e-1, 1, 5, 10, 20, 100]),
        'max_depth': hp.choice('max_depth', [2, 3, 4, 5, 7, 11])
    }

    best_result = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=100,
        trials=Trials(),
    )

    print(best_result)

In [8]:
hyperopt_search(train_data, valid_data, X_test, y_test, base_test)

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

2024/04/17 05:00:22 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logged at level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will u

Training until validation scores don't improve for 10 rounds                                                           
[50]	eval's auc: 0.811172                                                                                              
[100]	eval's auc: 0.822534                                                                                             
[150]	eval's auc: 0.826653                                                                                             
[200]	eval's auc: 0.828487                                                                                             
[250]	eval's auc: 0.829584                                                                                             
[300]	eval's auc: 0.830708                                                                                             
Early stopping, best iteration is:                                                                                     
[336]	eval's auc: 0.83113
  1%|▍        

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



Training until validation scores don't improve for 10 rounds                                                           
[50]	eval's auc: 0.8094                                                                                                
[100]	eval's auc: 0.821                                                                                                
[150]	eval's auc: 0.825435                                                                                             
[200]	eval's auc: 0.828287                                                                                             
[250]	eval's auc: 0.830057                                                                                             
[300]	eval's auc: 0.831163                                                                                             
[350]	eval's auc: 0.832083                                                                                             
[400]	eval's auc: 0.832614              

...

Training until validation scores don't improve for 10 rounds                                                           
[50]	eval's auc: 0.786176                                                                                              
[100]	eval's auc: 0.805125                                                                                             
[150]	eval's auc: 0.813351                                                                                             
[200]	eval's auc: 0.81783                                                                                              
[250]	eval's auc: 0.820909                                                                                             
[300]	eval's auc: 0.823124                                                                                             
[350]	eval's auc: 0.824769                                                                                             
[400]	eval's auc: 0.826091              

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



Training until validation scores don't improve for 10 rounds                                                           
[50]	eval's auc: 0.808836                                                                                              
[100]	eval's auc: 0.822603                                                                                             
[150]	eval's auc: 0.828429                                                                                             
[200]	eval's auc: 0.83158                                                                                              
[250]	eval's auc: 0.833728                                                                                             
[300]	eval's auc: 0.835081                                                                                             
[350]	eval's auc: 0.835903                                                                                             
[400]	eval's auc: 0.836415              

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



Training until validation scores don't improve for 10 rounds                                                           
[50]	eval's auc: 0.800124                                                                                              
[100]	eval's auc: 0.816697                                                                                             
[150]	eval's auc: 0.82394                                                                                              
[200]	eval's auc: 0.827747                                                                                             
[250]	eval's auc: 0.830232                                                                                             
[300]	eval's auc: 0.832125                                                                                             
[350]	eval's auc: 0.833152                                                                                             
[400]	eval's auc: 0.833883              

# Experiment 13
removing groups of multicorrelated features

In [4]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

In [6]:
(base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
cat_cols_base = list(X_train.select_dtypes("category").columns)
num_cols_base = list(X_train.select_dtypes(exclude="category").columns)
nans_df = X_train[num_cols_base].isna()
nans_groups={}
for col in num_cols_base:
    cur_group = nans_df[col].sum()
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]

In [7]:
nans_groups

{0: ['month_decision',
  'weekday_decision',
  'annuity_780A',
  'applicationcnt_361L',
  'applications30d_658L',
  'applicationscnt_1086L',
  'applicationscnt_464L',
  'applicationscnt_629L',
  'applicationscnt_867L',
  'clientscnt12m_3712952L',
  'clientscnt3m_3712950L',
  'clientscnt6m_3712949L',
  'clientscnt_100L',
  'clientscnt_1022L',
  'clientscnt_1071L',
  'clientscnt_1130L',
  'clientscnt_157L',
  'clientscnt_257L',
  'clientscnt_304L',
  'clientscnt_360L',
  'clientscnt_493L',
  'clientscnt_533L',
  'clientscnt_887L',
  'clientscnt_946L',
  'credamount_770A',
  'deferredmnthsnum_166L',
  'disbursedcredamount_1113A',
  'downpmt_116A',
  'homephncnt_628L',
  'mobilephncnt_593L',
  'numactivecreds_622L',
  'numactivecredschannel_414L',
  'numactiverelcontr_750L',
  'numcontrs3months_479L',
  'numnotactivated_1143L',
  'numpmtchanneldd_318L',
  'numrejects9m_859L',
  'sellerplacecnt_915L',
  'sellerplacescnt_216L',
  'max_mainoccupationinc_384A',
  'min_mainoccupationinc_384A',


In [11]:
def reduce_group(grps):
    """picks the features with the largest number of unique values from each group"""
    use = []
    for g in grps:
        mx = 0; vx = g[0]
        for gg in g:
            n = X_train[gg].nunique()
            if n>mx:
                mx = n
                vx = gg
        use.append(vx)
        #print()
    print('Use these',use)
    return use

def group_columns_by_correlation(matrix, threshold=0.8):
    """returns groups of correlated features"""
    correlation_matrix = matrix.corr()
    groups = []
    remaining_cols = list(matrix.columns)
    while remaining_cols:
        col = remaining_cols.pop(0)
        group = [col]
        correlated_cols = [col]
        for c in remaining_cols:
            if correlation_matrix.loc[col, c] >= threshold:
                group.append(c)
                correlated_cols.append(c)
        groups.append(group)
        remaining_cols = [c for c in remaining_cols if c not in correlated_cols]
    
    return groups

In [12]:
uses=[]
for k,v in nans_groups.items():
    if len(v)>1:
            Vs = nans_groups[k]
            #cross_features=list(combinations(Vs, 2))
            #make_corr(Vs)
            grps= group_columns_by_correlation(X_train[Vs], threshold=0.8)
            use=reduce_group(grps)
            uses=uses+use
            #make_corr(use)
    else:
        uses=uses+v
    print('####### NAN count =',k)
print(uses)
print(len(uses))
uses=uses+cat_cols_base
print(len(uses))
X_train=X_train[uses]

Use these ['month_decision', 'weekday_decision', 'credamount_770A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_867L', 'clientscnt_1022L', 'clientscnt_100L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'downpmt_116A', 'homephncnt_628L', 'mobilephncnt_593L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'sellerplacecnt_915L', 'max_mainoccupationinc_384A', 'max_birth_259D', 'max_num_group1_1_7', 'min_personindex_1023L', 'min_persontype_1072L', 'min_persontype_792L', 'first_personindex_1023L', 'first_persontype_1072L', 'first_persontype_792L', 'min_num_group1_1_7', 'first_num_group1_1_7']
####### 

In [13]:
uses

['month_decision',
 'weekday_decision',
 'credamount_770A',
 'applicationcnt_361L',
 'applications30d_658L',
 'applicationscnt_1086L',
 'applicationscnt_464L',
 'applicationscnt_867L',
 'clientscnt_1022L',
 'clientscnt_100L',
 'clientscnt_1071L',
 'clientscnt_1130L',
 'clientscnt_157L',
 'clientscnt_257L',
 'clientscnt_304L',
 'clientscnt_360L',
 'clientscnt_493L',
 'clientscnt_533L',
 'clientscnt_887L',
 'clientscnt_946L',
 'deferredmnthsnum_166L',
 'disbursedcredamount_1113A',
 'downpmt_116A',
 'homephncnt_628L',
 'mobilephncnt_593L',
 'numactivecreds_622L',
 'numactivecredschannel_414L',
 'numactiverelcontr_750L',
 'numcontrs3months_479L',
 'numnotactivated_1143L',
 'numpmtchanneldd_318L',
 'numrejects9m_859L',
 'sellerplacecnt_915L',
 'max_mainoccupationinc_384A',
 'max_birth_259D',
 'max_num_group1_1_7',
 'min_personindex_1023L',
 'min_persontype_1072L',
 'min_persontype_792L',
 'first_personindex_1023L',
 'first_persontype_1072L',
 'first_persontype_792L',
 'min_num_group1_1_7',


In [14]:
def experiment13(params) -> lgb.Booster:
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        eval_res = []
        test_auc = []
        test_gini = []
        for i in range(5):
            (base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
            cat_cols_base = list(X_train.select_dtypes("category").columns)
            X_train=X_train[uses]
            X_valid=X_valid[uses]
            X_test=X_test[uses]
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
            test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_cols_base)
            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            eval_res.append(eval_result['eval']['auc'][-1])
            ypred = bst.predict(X_test)
            base_test["score"] = ypred
            test_auc.append(roc_auc_score(y_test.values, ypred))
            test_gini.append(gini_stability(base_test))
        mlflow.log_metric("auc-eval", np.mean(eval_res))
        mlflow.log_metric("auc-test", np.mean(test_auc))
        mlflow.log_metric("gini-stability-test", np.mean(test_gini))
        mlflow.log_dict(
            {
                "eval_auc": eval_res,
                "test_auc": test_auc,
                "test_gini": test_gini,
                "description": "removing groups of multicorrelated features"
            },  
            "experiment13.json"
        )
        print("results: ", np.mean(eval_res), np.mean(test_auc), np.mean(test_gini))
        return bst, eval_res, test_auc, test_gini

In [15]:
bst, _, _, _ = experiment13(params)

2024/04/19 11:31:31 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logged at level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will us

Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.782319
[100]	eval's auc: 0.800979
[150]	eval's auc: 0.809299
[200]	eval's auc: 0.81443
[250]	eval's auc: 0.817712
[300]	eval's auc: 0.819799
[350]	eval's auc: 0.821408
[400]	eval's auc: 0.822727
[450]	eval's auc: 0.823947
[500]	eval's auc: 0.8248
[550]	eval's auc: 0.825554
[600]	eval's auc: 0.826266
[650]	eval's auc: 0.826867
[700]	eval's auc: 0.827381
[750]	eval's auc: 0.827726
[800]	eval's auc: 0.828249
[850]	eval's auc: 0.828645
Early stopping, best iteration is:
[865]	eval's auc: 0.828735


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.780786
[100]	eval's auc: 0.800297
[150]	eval's auc: 0.808762
[200]	eval's auc: 0.813554
[250]	eval's auc: 0.816786
[300]	eval's auc: 0.819086
[350]	eval's auc: 0.820928
[400]	eval's auc: 0.822412
[450]	eval's auc: 0.823726
[500]	eval's auc: 0.824657
[550]	eval's auc: 0.82554
[600]	eval's auc: 0.826206
[650]	eval's auc: 0.826594
[700]	eval's auc: 0.827219
[750]	eval's auc: 0.827629
[800]	eval's auc: 0.828053
[850]	eval's auc: 0.828417
[900]	eval's auc: 0.828752
[950]	eval's auc: 0.829113
[1000]	eval's auc: 0.829377
Did not meet early stopping. Best iteration is:
[1000]	eval's auc: 0.829377


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.778799
[100]	eval's auc: 0.798991
[150]	eval's auc: 0.80767
[200]	eval's auc: 0.813189
[250]	eval's auc: 0.816679
[300]	eval's auc: 0.819265
[350]	eval's auc: 0.821208
[400]	eval's auc: 0.822593
[450]	eval's auc: 0.823743
[500]	eval's auc: 0.824688
[550]	eval's auc: 0.825502
[600]	eval's auc: 0.826123
[650]	eval's auc: 0.826822
[700]	eval's auc: 0.827504
[750]	eval's auc: 0.828008
[800]	eval's auc: 0.828754
[850]	eval's auc: 0.82918
[900]	eval's auc: 0.829523
Early stopping, best iteration is:
[931]	eval's auc: 0.829613


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.785176
[100]	eval's auc: 0.803905
[150]	eval's auc: 0.812956
[200]	eval's auc: 0.817836
[250]	eval's auc: 0.821067
[300]	eval's auc: 0.82347
[350]	eval's auc: 0.825295
[400]	eval's auc: 0.826515
[450]	eval's auc: 0.827572
[500]	eval's auc: 0.828588
[550]	eval's auc: 0.829361
[600]	eval's auc: 0.83018
[650]	eval's auc: 0.830628
[700]	eval's auc: 0.831159
[750]	eval's auc: 0.831628
[800]	eval's auc: 0.832029
[850]	eval's auc: 0.832469
Early stopping, best iteration is:
[873]	eval's auc: 0.832707


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.782421
[100]	eval's auc: 0.803424
[150]	eval's auc: 0.812725
[200]	eval's auc: 0.817556
[250]	eval's auc: 0.821057
[300]	eval's auc: 0.823518
[350]	eval's auc: 0.825197
[400]	eval's auc: 0.826638
[450]	eval's auc: 0.827995
[500]	eval's auc: 0.828864
[550]	eval's auc: 0.829749
[600]	eval's auc: 0.830496
[650]	eval's auc: 0.831039
[700]	eval's auc: 0.831851
[750]	eval's auc: 0.832362
[800]	eval's auc: 0.832866
[850]	eval's auc: 0.833211
[900]	eval's auc: 0.833604
[950]	eval's auc: 0.833898
[1000]	eval's auc: 0.834182
Did not meet early stopping. Best iteration is:
[999]	eval's auc: 0.834186
results:  0.8309190676106857 0.8328988388075367 0.6290327368250018


In [16]:
params = {
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "auc",
        "max_depth": 11,
        "num_leaves": 48,
        "learning_rate": 0.05,
        "feature_fraction": 0.54,
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
        "n_estimators": 1000,
        "verbose": -1,
        "min_child_weight": 1.8,
        "reg_alpha": 5,
        "reg_lambda": 100,
        "colsample_bytree": 0.633,
        "feature_pre_filter": False,
        "min_child_samples": 237,
        "subsample": 0.52
    }

In [17]:
# repeat but with a different set of parameters - the best from hyperopt search
bst, _, _, _ = experiment13(params)

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.808484
[100]	eval's auc: 0.821165
[150]	eval's auc: 0.826807
[200]	eval's auc: 0.829862
[250]	eval's auc: 0.831866
[300]	eval's auc: 0.833114
[350]	eval's auc: 0.834068
[400]	eval's auc: 0.834982
[450]	eval's auc: 0.835553
[500]	eval's auc: 0.835893
[550]	eval's auc: 0.836138
Early stopping, best iteration is:
[566]	eval's auc: 0.836212


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.811013
[100]	eval's auc: 0.82347
[150]	eval's auc: 0.8288
[200]	eval's auc: 0.832038
[250]	eval's auc: 0.834152
[300]	eval's auc: 0.835518
[350]	eval's auc: 0.836472
[400]	eval's auc: 0.83717
[450]	eval's auc: 0.837669
[500]	eval's auc: 0.838085
[550]	eval's auc: 0.838345
[600]	eval's auc: 0.838547
Early stopping, best iteration is:
[616]	eval's auc: 0.838637


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.81032
[100]	eval's auc: 0.823273
[150]	eval's auc: 0.8292
[200]	eval's auc: 0.832494
[250]	eval's auc: 0.834744
[300]	eval's auc: 0.836073
[350]	eval's auc: 0.837051
[400]	eval's auc: 0.837929
Early stopping, best iteration is:
[393]	eval's auc: 0.837942


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.80553
[100]	eval's auc: 0.818919
[150]	eval's auc: 0.825579
[200]	eval's auc: 0.829024
[250]	eval's auc: 0.831284
[300]	eval's auc: 0.832955
[350]	eval's auc: 0.834253
[400]	eval's auc: 0.834935
[450]	eval's auc: 0.835569
[500]	eval's auc: 0.836049
[550]	eval's auc: 0.836398
[600]	eval's auc: 0.836642
[650]	eval's auc: 0.836846
Early stopping, best iteration is:
[668]	eval's auc: 0.83689


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.806997
[100]	eval's auc: 0.820112
[150]	eval's auc: 0.826319
[200]	eval's auc: 0.830185
[250]	eval's auc: 0.8324
[300]	eval's auc: 0.833982
[350]	eval's auc: 0.835203
[400]	eval's auc: 0.835875
[450]	eval's auc: 0.836473
[500]	eval's auc: 0.836797
[550]	eval's auc: 0.837129
Early stopping, best iteration is:
[584]	eval's auc: 0.837285
results:  0.8373766609327401 0.8354051780558052 0.633301657802139


# calculate psi for each feature

In [18]:
def psi(score_initial, score_new, num_bins = 10, mode = 'fixed'):
    
    eps = 1e-4
    
    # Sort the data
    score_initial.sort()
    score_new.sort()
    
    # Prepare the bins
    min_val = min(min(score_initial), min(score_new))
    max_val = max(max(score_initial), max(score_new))
    if mode == 'fixed':
        bins = [min_val + (max_val - min_val)*(i)/num_bins for i in range(num_bins+1)]
    elif mode == 'quantile':
        bins = pd.qcut(score_initial, q = num_bins, retbins = True)[1] # Create the quantiles based on the initial population
    else:
        raise ValueError(f"Mode \'{mode}\' not recognized. Your options are \'fixed\' and \'quantile\'")
    bins[0] = min_val - eps # Correct the lower boundary
    bins[-1] = max_val + eps # Correct the higher boundary
        
        
    # Bucketize the initial population and count the sample inside each bucket
    bins_initial = pd.cut(score_initial, bins = bins, labels = range(1,num_bins+1))
    df_initial = pd.DataFrame({'initial': score_initial, 'bin': bins_initial})
    grp_initial = df_initial.groupby('bin').count()
    grp_initial['percent_initial'] = grp_initial['initial'] / sum(grp_initial['initial'])
    
    # Bucketize the new population and count the sample inside each bucket
    bins_new = pd.cut(score_new, bins = bins, labels = range(1,num_bins+1))
    df_new = pd.DataFrame({'new': score_new, 'bin': bins_new})
    grp_new = df_new.groupby('bin').count()
    grp_new['percent_new'] = grp_new['new'] / sum(grp_new['new'])
    
    # Compare the bins to calculate PSI
    psi_df = grp_initial.join(grp_new, on = "bin", how = "inner")
    
    # Add a small value for when the percent is zero
    psi_df['percent_initial'] = psi_df['percent_initial'].apply(lambda x: eps if x == 0 else x)
    psi_df['percent_new'] = psi_df['percent_new'].apply(lambda x: eps if x == 0 else x)
    
    # Calculate the psi
    psi_df['psi'] = (psi_df['percent_initial'] - psi_df['percent_new']) * np.log(psi_df['percent_initial'] / psi_df['percent_new'])
    
    # Return the psi values
    return psi_df['psi'].values

In [20]:
csis = []
num_cols_base = list(X_train.select_dtypes(exclude="category").columns)
for col in num_cols_base:
    csi_values = psi(X_train[col].values, X_train[col].values, mode = 'quantile')
    csi = np.mean(csi_values)
    csis.append((csi,col))

ValueError: Bin edges must be unique: Index([1.0, 1.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0], dtype='float64').
You can drop duplicate edges by setting the 'duplicates' kwarg

# RFE

In [6]:
(base_train, X_train, y_train), (base_valid, X_valid, y_valid), (base_test, X_test, y_test) = dp.load_data_splits("data/train_base.parquet")
cat_cols_base = list(X_train.select_dtypes("category").columns)
num_cols_base = list(X_train.select_dtypes(exclude="category").columns)

In [7]:
uses = ['month_decision',
 'weekday_decision',
 'credamount_770A',
 'applicationcnt_361L',
 'applications30d_658L',
 'applicationscnt_1086L',
 'applicationscnt_464L',
 'applicationscnt_867L',
 'clientscnt_1022L',
 'clientscnt_100L',
 'clientscnt_1071L',
 'clientscnt_1130L',
 'clientscnt_157L',
 'clientscnt_257L',
 'clientscnt_304L',
 'clientscnt_360L',
 'clientscnt_493L',
 'clientscnt_533L',
 'clientscnt_887L',
 'clientscnt_946L',
 'deferredmnthsnum_166L',
 'disbursedcredamount_1113A',
 'downpmt_116A',
 'homephncnt_628L',
 'mobilephncnt_593L',
 'numactivecreds_622L',
 'numactivecredschannel_414L',
 'numactiverelcontr_750L',
 'numcontrs3months_479L',
 'numnotactivated_1143L',
 'numpmtchanneldd_318L',
 'numrejects9m_859L',
 'sellerplacecnt_915L',
 'max_mainoccupationinc_384A',
 'max_birth_259D',
 'max_num_group1_1_7',
 'min_personindex_1023L',
 'min_persontype_1072L',
 'min_persontype_792L',
 'first_personindex_1023L',
 'first_persontype_1072L',
 'first_persontype_792L',
 'min_num_group1_1_7',
 'first_num_group1_1_7',
 'assignmentdate_238D',
 'assignmentdate_4527235D',
 'pmtaverage_4527227A',
 'pmtcount_4527229L',
 'birthdate_574D',
 'contractssum_5085716L',
 'dateofbirth_337D',
 'days180_256L',
 'days30_165L',
 'days360_512L',
 'firstquarter_103L',
 'fourthquarter_440L',
 'secondquarter_766L',
 'thirdquarter_1082L',
 'pmtaverage_3A',
 'pmtcount_693L',
 'pmtscount_423L',
 'pmtssum_45A',
 'responsedate_1012D',
 'responsedate_4527233D',
 'responsedate_4917613D',
 'actualdpdtolerance_344P',
 'amtinstpaidbefduel24m_4187115A',
 'numinstlswithdpd5_4187116L',
 'annuitynextmonth_57A',
 'currdebt_22A',
 'currdebtcredtyperange_828A',
 'numinstls_657L',
 'totalsettled_863A',
 'mindbddpdlast24m_3658935P',
 'avgdbddpdlast3m_4187120P',
 'avgdbdtollast24m_4525197P',
 'avgdpdtolclosure24_3658938P',
 'avginstallast24m_3658937A',
 'maxinstallast24m_3658928A',
 'avglnamtstart24m_4525187A',
 'avgmaxdpdlast9m_3716943P',
 'avgoutstandbalancel6m_4187114A',
 'avgpmtlast12m_4525200A',
 'cntincpaycont9m_3716944L',
 'cntpmts24_3658933L',
 'commnoinclast6m_3546845L',
 'maxdpdfrom6mto36m_3546853P',
 'datefirstoffer_1144D',
 'datelastinstal40dpd_247D',
 'datelastunpaid_3546854D',
 'daysoverduetolerancedd_3976961L',
 'numinstpaidearly_338L',
 'numinstpaidearly5d_1087L',
 'numinstpaidlate1d_3546852L',
 'numinsttopaygr_769L',
 'dtlastpmtallstes_4499206D',
 'eir_270L',
 'firstclxcampaign_1125D',
 'firstdatedue_489D',
 'inittransactionamount_650A',
 'lastactivateddate_801D',
 'lastapplicationdate_877D',
 'min_creationdate_885D',
 'max_num_group1',
 'min_num_group1',
 'first_num_group1',
 'lastapprcredamount_781A',
 'lastapprdate_640D',
 'lastdelinqdate_224D',
 'lastrejectcredamount_222A',
 'lastrejectdate_50D',
 'maininc_215A',
 'mastercontrelectronic_519L',
 'mastercontrexist_109L',
 'maxannuity_159A',
 'maxdebt4_972A',
 'maxdpdlast24m_143P',
 'maxdpdlast3m_392P',
 'maxdpdtolerance_374P',
 'maxdbddpdlast1m_3658939P',
 'maxdbddpdtollast12m_3658940P',
 'maxdbddpdtollast6m_4187119P',
 'maxdpdinstldate_3546855D',
 'maxdpdinstlnum_3546846P',
 'maxlnamtstart6m_4525199A',
 'maxoutstandbalancel12m_4187113A',
 'maxpmtlast3m_4525190A',
 'numincomingpmts_3546848L',
 'numinstlsallpaid_934L',
 'numinstlswithdpd10_728L',
 'numinstlswithoutdpd_562L',
 'numinstpaid_4499208L',
 'numinstpaidearly3d_3546850L',
 'numinstregularpaidest_4493210L',
 'numinstpaidearly5dest_4493211L',
 'sumoutstandtotalest_4493215A',
 'numinstpaidlastcontr_4325080L',
 'numinstregularpaid_973L',
 'pctinstlsallpaidearl3d_427L',
 'pctinstlsallpaidlate1d_3546856L',
 'pctinstlsallpaidlat10d_839L',
 'pctinstlsallpaidlate4d_3546849L',
 'pctinstlsallpaidlate6d_3546844L',
 'pmtnum_254L',
 'posfpd10lastmonth_333P',
 'posfpd30lastmonth_3976960P',
 'posfstqpd30lastmonth_3976962P',
 'price_1097A',
 'sumoutstandtotal_3546847A',
 'totaldebt_9A',
 'totinstallast1m_4525188A',
 'validfrom_1069D',
 'mean_actualdpd_943P',
 'min_actualdpd_943P',
 'max_annuity_853A',
 'min_annuity_853A',
 'mean_annuity_853A',
 'mean_credacc_actualbalance_314A',
 'mean_credacc_maxhisbal_375A',
 'mean_credacc_minhisbal_90A',
 'min_credacc_maxhisbal_375A',
 'max_credacc_transactions_402L',
 'min_credacc_transactions_402L',
 'max_credacc_credlmt_575A',
 'max_credamount_590A',
 'max_downpmt_134A',
 'min_credacc_credlmt_575A',
 'min_credamount_590A',
 'min_downpmt_134A',
 'mean_credacc_credlmt_575A',
 'mean_credamount_590A',
 'mean_downpmt_134A',
 'max_currdebt_94A',
 'min_currdebt_94A',
 'mean_currdebt_94A',
 'max_mainoccupationinc_437A',
 'min_mainoccupationinc_437A',
 'mean_mainoccupationinc_437A',
 'mean_maxdpdtolerance_577P',
 'min_maxdpdtolerance_577P',
 'max_outstandingdebt_522A',
 'min_outstandingdebt_522A',
 'mean_outstandingdebt_522A',
 'mean_revolvingaccount_394A',
 'first_actualdpd_943P',
 'first_annuity_853A',
 'first_credacc_credlmt_575A',
 'first_credamount_590A',
 'first_downpmt_134A',
 'first_currdebt_94A',
 'first_mainoccupationinc_437A',
 'first_maxdpdtolerance_577P',
 'first_outstandingdebt_522A',
 'first_revolvingaccount_394A',
 'last_actualdpd_943P',
 'last_annuity_853A',
 'last_credacc_actualbalance_314A',
 'last_credacc_maxhisbal_375A',
 'last_credacc_minhisbal_90A',
 'last_credacc_transactions_402L',
 'last_credacc_credlmt_575A',
 'last_credamount_590A',
 'last_downpmt_134A',
 'last_currdebt_94A',
 'last_mainoccupationinc_437A',
 'last_maxdpdtolerance_577P',
 'last_outstandingdebt_522A',
 'max_approvaldate_319D',
 'min_approvaldate_319D',
 'max_dateactivated_425D',
 'min_dateactivated_425D',
 'max_dtlastpmt_581D',
 'min_dtlastpmt_581D',
 'max_dtlastpmtallstes_3545839D',
 'min_dtlastpmtallstes_3545839D',
 'min_employedfrom_700D',
 'max_firstnonzeroinstldate_307D',
 'min_firstnonzeroinstldate_307D',
 'first_approvaldate_319D',
 'first_dateactivated_425D',
 'first_dtlastpmt_581D',
 'first_dtlastpmtallstes_3545839D',
 'first_employedfrom_700D',
 'first_firstnonzeroinstldate_307D',
 'last_approvaldate_319D',
 'last_creationdate_885D',
 'last_dateactivated_425D',
 'last_dtlastpmt_581D',
 'last_dtlastpmtallstes_3545839D',
 'last_employedfrom_700D',
 'last_firstnonzeroinstldate_307D',
 'max_byoccupationinc_3656910L',
 'min_byoccupationinc_3656910L',
 'max_childnum_21L',
 'min_childnum_21L',
 'max_pmtnum_8L',
 'min_pmtnum_8L',
 'first_childnum_21L',
 'first_pmtnum_8L',
 'last_byoccupationinc_3656910L',
 'last_childnum_21L',
 'last_pmtnum_8L',
 'mean_amount_4527230A',
 'min_amount_4527230A',
 'first_amount_4527230A',
 'last_amount_4527230A',
 'max_recorddate_4527225D',
 'max_num_group1_1_3',
 'min_num_group1_1_3',
 'first_num_group1_1_3',
 'last_num_group1_1_3',
 'mean_amount_4917619A',
 'min_amount_4917619A',
 'first_amount_4917619A',
 'last_amount_4917619A',
 'max_deductiondate_4917603D',
 'min_deductiondate_4917603D',
 'first_deductiondate_4917603D',
 'last_deductiondate_4917603D',
 'mean_deductiondate_4917603D',
 'max_num_group1_1_4',
 'min_num_group1_1_4',
 'first_num_group1_1_4',
 'last_num_group1_1_4',
 'mean_pmtamount_36A',
 'min_pmtamount_36A',
 'first_pmtamount_36A',
 'last_pmtamount_36A',
 'max_processingdate_168D',
 'min_processingdate_168D',
 'first_processingdate_168D',
 'last_processingdate_168D',
 'max_num_group1_1_5',
 'min_num_group1_1_5',
 'first_num_group1_1_5',
 'last_num_group1_1_5',
 'last_mainoccupationinc_384A',
 'last_birth_259D',
 'max_empl_employedfrom_271D',
 'last_personindex_1023L',
 'last_persontype_1072L',
 'mean_amount_416A',
 'last_amount_416A',
 'min_openingdate_313D',
 'max_num_group1_1_8',
 'min_num_group1_1_8',
 'first_num_group1_1_8',
 'last_num_group1_1_8',
 'min_openingdate_857D',
 'first_openingdate_857D',
 'last_openingdate_857D',
 'max_num_group1_1_9',
 'min_num_group1_1_9',
 'first_num_group1_1_9',
 'last_num_group1_1_9',
 'description_5085714M',
 'education_1103M',
 'education_88M',
 'maritalst_385M',
 'maritalst_893M',
 'requesttype_4525192L',
 'bankacctype_710L',
 'cardtype_51L',
 'credtype_322L',
 'disbursementtype_67L',
 'equalitydataagreement_891L',
 'inittransactioncode_186L',
 'isbidproduct_1095L',
 'isdebitcard_729L',
 'lastapprcommoditycat_1041M',
 'lastcancelreason_561M',
 'lastrejectcommoditycat_161M',
 'lastrejectcommodtypec_5251769M',
 'lastrejectreason_759M',
 'lastrejectreasonclient_4145040M',
 'lastst_736L',
 'opencred_647L',
 'paytype1st_925L',
 'paytype_783L',
 'twobodfilling_608L',
 'typesuite_864L',
 'max_cancelreason_3545846M',
 'max_education_1138M',
 'max_postype_4733339M',
 'max_rejectreason_755M',
 'max_rejectreasonclient_4145042M',
 'min_cancelreason_3545846M',
 'min_education_1138M',
 'min_postype_4733339M',
 'min_rejectreason_755M',
 'min_rejectreasonclient_4145042M',
 'first_cancelreason_3545846M',
 'first_education_1138M',
 'first_postype_4733339M',
 'first_rejectreason_755M',
 'first_rejectreasonclient_4145042M',
 'last_cancelreason_3545846M',
 'last_education_1138M',
 'last_postype_4733339M',
 'last_rejectreason_755M',
 'last_rejectreasonclient_4145042M',
 'mode_cancelreason_3545846M',
 'mode_education_1138M',
 'mode_postype_4733339M',
 'mode_rejectreason_755M',
 'mode_rejectreasonclient_4145042M',
 'max_credacc_status_367L',
 'max_credtype_587L',
 'max_familystate_726L',
 'max_inittransactioncode_279L',
 'max_isbidproduct_390L',
 'max_isdebitcard_527L',
 'max_status_219L',
 'min_credacc_status_367L',
 'min_credtype_587L',
 'min_familystate_726L',
 'min_inittransactioncode_279L',
 'min_isbidproduct_390L',
 'min_isdebitcard_527L',
 'min_status_219L',
 'first_credtype_587L',
 'first_familystate_726L',
 'first_inittransactioncode_279L',
 'first_isbidproduct_390L',
 'first_isdebitcard_527L',
 'first_status_219L',
 'last_credacc_status_367L',
 'last_credtype_587L',
 'last_familystate_726L',
 'last_inittransactioncode_279L',
 'last_isbidproduct_390L',
 'last_status_219L',
 'max_education_927M',
 'max_empladdr_district_926M',
 'max_empladdr_zipcode_114M',
 'max_language1_981M',
 'min_education_927M',
 'min_language1_981M',
 'first_education_927M',
 'first_language1_981M',
 'last_education_927M',
 'last_empladdr_district_926M',
 'last_empladdr_zipcode_114M',
 'last_language1_981M',
 'mode_education_927M',
 'mode_language1_981M',
 'max_contaddr_matchlist_1032L',
 'max_contaddr_smempladdr_334L',
 'max_empl_employedtotal_800L',
 'max_empl_industry_691L',
 'max_familystate_447L',
 'max_housetype_905L',
 'max_incometype_1044T',
 'max_relationshiptoclient_415T',
 'max_relationshiptoclient_642T',
 'max_remitter_829L',
 'max_role_1084L',
 'max_safeguarantyflag_411L',
 'max_sex_738L',
 'max_type_25L',
 'min_contaddr_matchlist_1032L',
 'min_contaddr_smempladdr_334L',
 'min_empl_employedtotal_800L',
 'min_empl_industry_691L',
 'min_familystate_447L',
 'min_housetype_905L',
 'min_incometype_1044T',
 'min_relationshiptoclient_415T',
 'min_relationshiptoclient_642T',
 'min_remitter_829L',
 'min_safeguarantyflag_411L',
 'min_sex_738L',
 'min_type_25L',
 'first_contaddr_matchlist_1032L',
 'first_contaddr_smempladdr_334L',
 'first_empl_employedtotal_800L',
 'first_empl_industry_691L',
 'first_familystate_447L',
 'first_housetype_905L',
 'first_incometype_1044T',
 'first_role_1084L',
 'first_safeguarantyflag_411L',
 'first_sex_738L',
 'first_type_25L',
 'last_contaddr_matchlist_1032L',
 'last_contaddr_smempladdr_334L',
 'last_incometype_1044T',
 'last_relationshiptoclient_415T',
 'last_relationshiptoclient_642T',
 'last_remitter_829L',
 'last_role_1084L',
 'last_safeguarantyflag_411L',
 'last_sex_738L',
 'last_type_25L']

In [17]:
to_drop = []
for col in uses:
    if X_train[col].isnull().astype(int).sum() > 0.6 * len(X_train[col]):
        to_drop.append(col)

In [18]:
uses = [x for x in uses if x not in to_drop]
len(uses)

311

In [20]:
X_train=X_train[uses]
X_valid=X_valid[uses]
X_test=X_test[uses]

In [21]:
CREDIT_EXPERIMENT_NAME = "credit-score-rfe"
EXPERIMENT_NAME = "chosen-models-credit"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [25]:
def rfe_cat(tol = 1e-3):
    with mlflow.start_run():
        mlflow.set_tag("model", "lgb")
        mlflow.log_params(params)
        eval_result = {}
        cat_cols_base = list(X_train.select_dtypes("category").columns)
        to_keep = list(X_train.columns)
        train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols_base)
        valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols_base)
        bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                        callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
        ypred = bst.predict(X_test)
        keep_score_test = roc_auc_score(y_test.values, ypred)
        mlflow.log_metric("auc-eval", eval_result['eval']['auc'][-1])
        mlflow.log_metric("auc-test", keep_score_test)
        for col in X_train.columns:
            print(f"try: {col}")
            to_train = [x for x in to_keep if x != col]
            cat_cols_base = list(X_train[to_train].select_dtypes("category").columns)
            train_data = lgb.Dataset(X_train[to_train], label=y_train, categorical_feature=cat_cols_base)
            valid_data = lgb.Dataset(X_valid[to_train], label=y_valid, categorical_feature=cat_cols_base)
            bst = lgb.train(params, train_data, valid_sets=valid_data, valid_names=["eval"], 
                            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10), lgb.record_evaluation(eval_result)])
            ypred = bst.predict(X_test[to_train])
            score_test = roc_auc_score(y_test.values, ypred)
            mlflow.log_metric("auc-eval", eval_result['eval']['auc'][-1])
            mlflow.log_metric("auc-test", score_test)
            if score_test + tol > keep_score_test:
                to_keep = to_train
                keep_score_test = score_test
                print(f"removing {col}")
    return to_keep

In [26]:
rfe_cat()

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.776785
[100]	eval's auc: 0.795294
[150]	eval's auc: 0.803044
[200]	eval's auc: 0.807698
[250]	eval's auc: 0.810651
[300]	eval's auc: 0.813102
[350]	eval's auc: 0.81443
[400]	eval's auc: 0.815832
[450]	eval's auc: 0.816773
[500]	eval's auc: 0.817792
[550]	eval's auc: 0.818294
[600]	eval's auc: 0.818928
[650]	eval's auc: 0.819383
[700]	eval's auc: 0.819855
[750]	eval's auc: 0.820425
Early stopping, best iteration is:
[768]	eval's auc: 0.820666
try: month_decision


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.776699
[100]	eval's auc: 0.795259
[150]	eval's auc: 0.803514
[200]	eval's auc: 0.807921
[250]	eval's auc: 0.811002
[300]	eval's auc: 0.813207
[350]	eval's auc: 0.814506
[400]	eval's auc: 0.815785
[450]	eval's auc: 0.816743
[500]	eval's auc: 0.817826
[550]	eval's auc: 0.818535
[600]	eval's auc: 0.819046
[650]	eval's auc: 0.819513
[700]	eval's auc: 0.819846
[750]	eval's auc: 0.820293
[800]	eval's auc: 0.820667
[850]	eval's auc: 0.821037
[900]	eval's auc: 0.821261
[950]	eval's auc: 0.8215
Early stopping, best iteration is:
[979]	eval's auc: 0.82171
removing month_decision
try: weekday_decision


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.769956
[100]	eval's auc: 0.786981
[150]	eval's auc: 0.794414
[200]	eval's auc: 0.798749
[250]	eval's auc: 0.80167
[300]	eval's auc: 0.803925
[350]	eval's auc: 0.805721
[400]	eval's auc: 0.807197
[450]	eval's auc: 0.808138
[500]	eval's auc: 0.808798
[550]	eval's auc: 0.809449
[600]	eval's auc: 0.810213
[650]	eval's auc: 0.810547
[700]	eval's auc: 0.811103
Early stopping, best iteration is:
[720]	eval's auc: 0.811299
try: last_type_25L


C:\Users\wojtek\anaconda3\envs\py311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	eval's auc: 0.769956
[100]	eval's auc: 0.786981
[150]	eval's auc: 0.794414
[200]	eval's auc: 0.798749
[250]	eval's auc: 0.80167
[300]	eval's auc: 0.803925
[350]	eval's auc: 0.805722
[400]	eval's auc: 0.807184
[450]	eval's auc: 0.80814
[500]	eval's auc: 0.808937
[550]	eval's auc: 0.809552
[600]	eval's auc: 0.810361
[650]	eval's auc: 0.81081
[700]	eval's auc: 0.811166
Early stopping, best iteration is:
[721]	eval's auc: 0.811367
removing last_type_25L


['credamount_770A',
 'applications30d_658L',
 'applicationscnt_1086L',
 'applicationscnt_464L',
 'applicationscnt_867L',
 'clientscnt_1022L',
 'clientscnt_100L',
 'clientscnt_1130L',
 'clientscnt_533L',
 'clientscnt_887L',
 'disbursedcredamount_1113A',
 'mobilephncnt_593L',
 'numpmtchanneldd_318L',
 'dateofbirth_337D',
 'days180_256L',
 'days30_165L',
 'thirdquarter_1082L',
 'responsedate_4527233D',
 'actualdpdtolerance_344P',
 'amtinstpaidbefduel24m_4187115A',
 'annuitynextmonth_57A',
 'currdebt_22A',
 'currdebtcredtyperange_828A',
 'avgoutstandbalancel6m_4187114A',
 'eir_270L',
 'firstdatedue_489D',
 'lastactivateddate_801D',
 'min_creationdate_885D',
 'lastrejectdate_50D',
 'mastercontrelectronic_519L',
 'mastercontrexist_109L',
 'maxdebt4_972A',
 'maxdpdlast24m_143P',
 'maxdpdlast3m_392P',
 'maxdbddpdtollast12m_3658940P',
 'maxdbddpdtollast6m_4187119P',
 'maxdpdinstldate_3546855D',
 'numinstlswithoutdpd_562L',
 'pctinstlsallpaidlate4d_3546849L',
 'posfstqpd30lastmonth_3976962P',
 '

In [27]:
to_keep = ['credamount_770A',
 'applications30d_658L',
 'applicationscnt_1086L',
 'applicationscnt_464L',
 'applicationscnt_867L',
 'clientscnt_1022L',
 'clientscnt_100L',
 'clientscnt_1130L',
 'clientscnt_533L',
 'clientscnt_887L',
 'disbursedcredamount_1113A',
 'mobilephncnt_593L',
 'numpmtchanneldd_318L',
 'dateofbirth_337D',
 'days180_256L',
 'days30_165L',
 'thirdquarter_1082L',
 'responsedate_4527233D',
 'actualdpdtolerance_344P',
 'amtinstpaidbefduel24m_4187115A',
 'annuitynextmonth_57A',
 'currdebt_22A',
 'currdebtcredtyperange_828A',
 'avgoutstandbalancel6m_4187114A',
 'eir_270L',
 'firstdatedue_489D',
 'lastactivateddate_801D',
 'min_creationdate_885D',
 'lastrejectdate_50D',
 'mastercontrelectronic_519L',
 'mastercontrexist_109L',
 'maxdebt4_972A',
 'maxdpdlast24m_143P',
 'maxdpdlast3m_392P',
 'maxdbddpdtollast12m_3658940P',
 'maxdbddpdtollast6m_4187119P',
 'maxdpdinstldate_3546855D',
 'numinstlswithoutdpd_562L',
 'pctinstlsallpaidlate4d_3546849L',
 'posfstqpd30lastmonth_3976962P',
 'min_annuity_853A',
 'mean_annuity_853A',
 'min_credamount_590A',
 'max_currdebt_94A',
 'min_currdebt_94A',
 'mean_currdebt_94A',
 'max_mainoccupationinc_437A',
 'min_mainoccupationinc_437A',
 'min_maxdpdtolerance_577P',
 'min_outstandingdebt_522A',
 'first_currdebt_94A',
 'first_mainoccupationinc_437A',
 'last_downpmt_134A',
 'last_maxdpdtolerance_577P',
 'min_dateactivated_425D',
 'max_dtlastpmtallstes_3545839D',
 'first_dtlastpmtallstes_3545839D',
 'first_firstnonzeroinstldate_307D',
 'last_employedfrom_700D',
 'max_childnum_21L',
 'max_pmtnum_8L',
 'last_childnum_21L',
 'description_5085714M',
 'education_1103M',
 'lastcancelreason_561M',
 'lastrejectcommoditycat_161M',
 'lastrejectreason_759M',
 'paytype_783L',
 'max_rejectreasonclient_4145042M',
 'min_cancelreason_3545846M',
 'min_education_1138M',
 'min_postype_4733339M',
 'min_rejectreason_755M',
 'min_rejectreasonclient_4145042M',
 'first_cancelreason_3545846M',
 'first_education_1138M',
 'first_rejectreason_755M',
 'first_rejectreasonclient_4145042M',
 'last_education_1138M',
 'last_rejectreason_755M',
 'last_rejectreasonclient_4145042M',
 'min_credtype_587L',
 'min_inittransactioncode_279L',
 'min_status_219L',
 'first_credtype_587L',
 'first_inittransactioncode_279L',
 'first_isbidproduct_390L',
 'first_status_219L',
 'last_credtype_587L',
 'last_familystate_726L',
 'last_inittransactioncode_279L',
 'last_isbidproduct_390L',
 'max_empladdr_district_926M',
 'max_empladdr_zipcode_114M',
 'min_education_927M',
 'min_language1_981M',
 'first_education_927M',
 'first_language1_981M',
 'last_education_927M',
 'last_empladdr_district_926M',
 'last_empladdr_zipcode_114M',
 'last_language1_981M',
 'mode_education_927M',
 'mode_language1_981M',
 'max_contaddr_matchlist_1032L',
 'max_contaddr_smempladdr_334L',
 'max_familystate_447L',
 'max_incometype_1044T',
 'max_role_1084L',
 'max_safeguarantyflag_411L',
 'max_sex_738L',
 'max_type_25L',
 'min_contaddr_matchlist_1032L',
 'min_contaddr_smempladdr_334L',
 'min_familystate_447L',
 'min_incometype_1044T',
 'min_safeguarantyflag_411L',
 'min_type_25L',
 'first_contaddr_smempladdr_334L',
 'first_familystate_447L',
 'first_incometype_1044T',
 'first_role_1084L',
 'first_safeguarantyflag_411L',
 'first_sex_738L',
 'first_type_25L',
 'last_contaddr_matchlist_1032L',
 'last_contaddr_smempladdr_334L',
 'last_incometype_1044T',
 'last_role_1084L',
 'last_sex_738L']

In [28]:
len(to_keep)

130

In [29]:
base_test["score"] = ypred
gini_stability(base_test)

NameError: name 'ypred' is not defined